目标检测的边框损失函数
===
IOU(Intersection Over Union)在目标检测中是一个非常重要的指标，NMS(Non-Max Supression非极大值抑制)是过滤重复预测框的非常重要的一个办法

# 1.Smooth L1损失函数

## 1.1.Smooth L1损失函数介绍

假设x为预测框和真实框之间的数值差异，常用的L1和L2 Loss定义为
$$
\begin{cases}
L_1=\left| x \right| \frac{dL_2(x)}{x}=2x \\
L_2=x^2 \\
Smooth_{L_1}=\begin{cases}
0.5x^2 & \left| x \right| < 1 \\
\left| x \right| -0.5 & otherwise
\end{cases}
\end{cases}
$$

## 1.2.损失函数的导数

$$
\frac{dL_1(x)}{x}=
\begin{cases}
1 & x \geq 0 \\
-1 & otherwise
\end{cases}
$$

$$\frac{dL_2(x)}{x}=2x$$

$$
\frac{dSmooth_{L_1}(x)}{x}=
\begin{cases}
x & \left| x \right| < 1 \\
\pm{1} & otherwise
\end{cases}
$$

## 1.3. 损失函数总结
从损失函数对x的导数可知： $L_1$损失函数对x的导数为常数，在训练后期，x很小时，如果learning rate 不变，损失函数会在稳定值附近波动，很难收敛到更高的精度。 $L_2$损失函数对x的导数在x值很大时，其导数也非常大，在训练初期不稳定。$Smooth_{L_1}$完美的避开了 $L_1$和$L_2$损失的缺点。

## 1.4. 实际目标检测框回归任务中的损失loss

$$L_{loc}(t^u, v)=\sum_{i \in \{x,y,w,h\}}Smooth_{L_1}(t_i^u-v_i)$$

其中$v=(v_x,v_y,v_w,v_h)$表示GT的框坐标，$t^u=(t_x^u,t_y^u,t_w^u,t_h^u)$表示预测的框坐标，即分别求4个点的loss，然后相加作为Bounding Box Regression Loss。

![Images](...%0AQf/Z)

## 1.5.$Smooth_{L_1}$的缺点
- 上面的三种Loss用于计算目标检测的Bounding Box Loss时，独立的求出4个点的Loss，然后进行相加得到最终的Bounding Box Loss，这种做法的假设是4个点是相互独立的，实际是有一定相关性的
- 实际评价框检测的指标是使用IOU，这两者是不等价的，多个检测框可能有相同大小的$Smooth_{L_1}(x)$ Loss，但IOU可能差异很大，为了解决这个问题就引入了IOU LOSS。

# 2.IOU

## 2.1.通过4个坐标点独立回归Building boxes的缺点

- 检测评价的方式是使用IoU,而实际回归坐标框的时候是使用4个坐标点，如下图所示，是不等价的；L1或者L2 Loss相同的框，其IoU 不是唯一的
- 通过4个点回归坐标框的方式是假设4个坐标点是相互独立的，没有考虑其相关性，实际4个坐标点具有一定的相关性
- 基于L1和L2的距离的loss对于尺度不具有不变性

## 2.2.IOU所引起的问题
![image](...%0AiD//2Q==)

图(a)中的三组框具有相同的L2 Loss，但其IoU差异很大；图(b)中的三组框具有相同的L1 Loss,但IoU 同样差异很大，说明L1,L2这些Loss用于回归任务时，不能等价于最后用于评测检测的IoU.

IoU 是目标检测领域最重要的评价尺度之一，特性是对尺度不敏感，主要判断检测框的重合程度。但是对于CNN而言，没有方向信息，无法反馈神经网络的边界回归框应该如何调整。IoU作为度量和损失都有两个主要问题
- 如果两个对象不重叠，IoU值将为零，并且不能反映两个形状之间的距离。在非重叠对象的情况下，如果使用IoU作为损失，则梯度为零，无法优化
- IoU不能正确区分两个对象的不同对齐。更精确地说，对于两个具有相同交点水平的不同方向重叠的对象，IoU将是完全相等，因此，IoU函数的值并不能反映两个对象是如何重叠的。

![image](...)

于是通过将IoU的概念扩展到非重叠的情况来解决IoU的这两个弱点。这个推广遵循三个条件
- 与IOU相同的定义，即将比较对象的形状属性编码到区域属性中
- 保持IOU的尺度不变性质
- 确保在发生重叠物体时与IOU有很强的相关性

## 2.3.IOU介绍
在目标检测的评价体系中，有一个参数叫做IoU，简单来讲就是模型产生的目标窗口和原来标记窗口的交叠率。具体我们可以简单的理解为检测结果(DetectionResult)与Ground Truth的交集比上它们的并集，即为检测的准确率IoU
$$IOU=\frac{DR \cap GT}{DR \cup GT}$$

![image](...)

GT = GroundTruth; DR = DetectionResult; 黄色边框框起来的是$GT \cap DR$，绿色框框起来的是$GT \cup DR$,当然最理想的情况就是 DR 与 GT 完全重合，即IOU=1

![image](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAlgAAAHUCAYAAADm/FbiAAAAGXRFWHRTb2Z0%0Ad2FyZQBBZG9iZSBJbWFnZVJlYWR5ccllPAAAA3ppVFh0WE1MOmNvbS5hZG9i%0AZS54bXAAAAAAADw/eHBhY2tldCBiZWdpbj0i77u/IiBpZD0iVzVNME1wQ2Vo%0AaUh6cmVTek5UY3prYzlkIj8+IDx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6%0AbnM6bWV0YS8iIHg6eG1wdGs9IkFkb2JlIFhNUCBDb3JlIDUuNi1jMTMyIDc5%0ALjE1OTI4NCwgMjAxNi8wNC8xOS0xMzoxMzo0MCAgICAgICAgIj4gPHJkZjpS%0AREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJk%0AZi1zeW50YXgtbnMjIj4gPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIg%0AeG1sbnM6eG1wTU09Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC9tbS8i%0AIHhtbG5zOnN0UmVmPSJodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvc1R5%0AcGUvUmVzb3VyY2VSZWYjIiB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j%0Ab20veGFwLzEuMC8iIHhtcE1NOk9yaWdpbmFsRG9jdW1lbnRJRD0ieG1wLmRp%0AZDpmZGYyYWU2MC05MWU3LTRiZTgtYjNlMC1mYmMzZDFlMzQ0NDEiIHhtcE1N%0AOkRvY3VtZW50SUQ9InhtcC5kaWQ6RjVCMzEzODU3QzMzMTFFNjlFRjhGMDJB%0AOTBFQjRGM0QiIHhtcE1NOkluc3RhbmNlSUQ9InhtcC5paWQ6RjVCMzEzODQ3%0AQzMzMTFFNjlFRjhGMDJBOTBFQjRGM0QiIHhtcDpDcmVhdG9yVG9vbD0iQWRv%0AYmUgUGhvdG9zaG9wIENDIDIwMTUuNSAoTWFjaW50b3NoKSI+IDx4bXBNTTpE%0AZXJpdmVkRnJvbSBzdFJlZjppbnN0YW5jZUlEPSJ4bXAuaWlkOmZkZjJhZTYw%0ALTkxZTctNGJlOC1iM2UwLWZiYzNkMWUzNDQ0MSIgc3RSZWY6ZG9jdW1lbnRJ%0ARD0ieG1wLmRpZDpmZGYyYWU2MC05MWU3LTRiZTgtYjNlMC1mYmMzZDFlMzQ0%0ANDEiLz4gPC9yZGY6RGVzY3JpcHRpb24+IDwvcmRmOlJERj4gPC94OnhtcG1l%0AdGE+IDw/eHBhY2tldCBlbmQ9InIiPz4bOXKGAAAniklEQVR42uzdC7TdVX0n%0A8H3zAvKAYIHmMWLCQ0CgCsNA8YmRslzMYKmKj9qglMbisogFFiyzOgVmdWDh%0AMoxUxRoHU82oKAbTUClGYKwIFMYZLIESEwTEpgRCSSCXAHmdub8T///sc3Lu%0Avefce87NfXw+a13uOfeex//s/5/s7/3t/d//rkql8mRK6YAEAEA7vNDVE7C2%0A9dyYoC0AANpi+7ie/3RrBwCAtukepw0AANpLwAIAELAAAAQsAAABCwAAAQsA%0AQMACABCwAAAQsAAA9gaXyAEY41Y8+Upa/vjL6dfdO1JXl/agPSqVnpAxLqWT%0ADpmUPnTEfunY10wUsAAYG57qCVW3/eqV9JOnX03d2yoahLZ7cWsl/da+48Zc%0AwIqLPW/s+T7dIQAwtjzw7Nb03//v5nT/M1s1Bh133VsOSEccMCGdfMiksfBx%0AN6lgAYxR3/jFljJcjetKad/xxgdpr52VlF7Zsasy+rVHt6SjDxwzAcsQIcBY%0AtWbT9vL2kQdMSMve/VtlpwiDFaF91b9vSx/+0fPV+09s3l792VghYAGMUXmQ%0A2md8VzpoXyeW016vnTq+vB2T3sdSePd/E8AYlVcTVK3ohJeyEyfieBtLFSwB%0ACwBAwAIAELAAAAQsAAAELAAAAQsAQMACAEDAAgAQsAAABCwAAAQsAAABCwBA%0AwAIAELAAABCwAAAELAAAAQsAAAELAEDAAgAQsAAAELAAAAQsAAABCwAAAQsA%0AQMACABCwAAAELAAABCwAAAELAEDAAgBAwAIAELAAAAQsAAAELAAAAQsAQMAC%0AAGCXCZoAgJFoy/ZKuvHRl9IPn3o1vbB1pwYZhl7dUanZX7/YtD297fsbRsS2%0ATxzXlU6bPSl97Ogp6dCp4wUsAEa/p7p3pAvv3pTuf2arxhghdlZ2Ba61L2wf%0AMdv8Lxu3pW+ueTn96D0HtRyyBCwARpy/Xf3S7nDV1RX/0SjDVk+yqlRG7NZH%0AdTSOt788aX8BC4DR7Z71uytXUyaMSy9tr2iUYaurGrL2Hd+VfnvyuPRf/+P+%0AI+QYezUtWb1lj+NNwAJg1Mrn9uw7oUvAGgF6cnA6YNK49F/m7Dsitne/nuOq%0ACFj58dYsZxECMOJM7No9JLizIlzRfs+9srPh8SZgAQDsJQIWAICABQAgYAEA%0ACFgAAAhYAAACFgCAgAUAgIAFACBgAQAIWAAACFgAAAIWAICABQAgYAEAIGAB%0AAAhYAAACFgAAAhYAgIAFACBgAQAgYAEACFgAAAIWAAACFgCAgAUAIGABAAhY%0AAAAIWAAAAhYAgIAFAICABQAgYAEACFgAAAhYAAACFgCAgAUAgIAFACBgAQAI%0AWAAAAhYAAAIWAICABQAgYAEAIGABAAhYAAACFgAAAhYAgIAFACBgAQAgYAEA%0ACFgAAAIWAICABQCAgAUAIGABAAhYAAAIWAAAAhYAgIAFAICABQAgYAEACFgA%0AAAIWAAACFgCAgAUAIGABACBgAQAIWAAAAhYAAAIWAICABQAgYAEAIGABAAhY%0AAAACFgCAgAUAgIAFACBgAQAIWAAACFgAAAIWAICABQCAgAUAIGABAAhYAAAI%0AWAAAAhYAgIAFACBgAQAgYAEACFgAAAIWAAACFgCAgAUAIGABACBgAQAIWAAA%0AAhYAAAIWAICABQAgYAEACFgAAAhYALSsUtl9u6tLe9BZOyu1x9xoN8EuB2DH%0AzpRe3VEpO8Jh31nnQdHuQ8ACYLiYu//4tOr5bdXba17Ylk7+3rMjJqw8/+ru%0AiLV5ayWN60rVL4ZfcK/85qg6dNr46jEnYAEwqp171JT0VPeO9PPntqXtPR3h%0AMy/vHKGd+K7t3mmXDmsf7Tnejjxg7MQOc7AAxqi3zpyU/sdbpqd3zNpHY9Ax%0AB+83Ll1w7JT0saMnV4+5sUIFC2AMO+bACekjr5+cpkzoSr/u3jFiJruvfWF7%0Aenn7rqGnA/cZl147dbydOczEhPYJ41I66ZBJ6ZzD90tjbQRXwAIY494zZ9/q%0A10jyeyueK+ePnfHafdL1b51uRzKsGCIEABCwAAAELAAAAQsAAAELAEDAAgAQ%0AsAAAELAAAAQsAAABCwAAAQsAQMACABhZXOwZRrEtW7akJ554srx/yCEHp4MP%0APljDAAhYDLX7738g3XLL8vL+qaf+bjr77PdomA619V13/Th99atfr96fPXtm%0AuvLKheld75o34Nd85JF/ST/+8U/STTd9L61b93TDxyxY8NF08sknDep9hrM7%0A77wrrVx5R/X2GWecPmo/JyBgMYJEh3/zzX9X3o/b0UlNnjxZ47TR8uUr0sUX%0Af6bmZxGINm/uHtDrPfnkr9KXv/yVmn3Xmwh08XXCCcenyy67OJ1yysmjqm2j%0ADYt2iD8QAAQs9qoNGzaU1ZTcfff9kypAm9s5D1dRuXrzmwcecqJic+WVV+9R%0AsTrnnN+vCRiPPro63XbbyvJxDz64Kn34w+elq65amObP/4gdAyBg0Qn33HNf%0ATedcVAG+853vCVgdaueoIi1d+rUBVwgjXC1YcGFNWLvkkk81rDrGUO9nPnNZ%0A9Tk33LC4GrDCFVdcnV58cXP65CcvsHMA2sBZhNRYuvRb5e1LL/3zamcd7rjj%0AH6tDULTf/Pl/OOBwFfskKld5uFq69MZqkOrrNSMsR6g7/fR3lD9btOgL1WFL%0AAAQs2igmRxcVjeh442yzD33o/eXv7777pxppmLn66mvL4b4iXM2Z87qmnhsB%0A7POf/1xdyPrr6vAlAAIWbRJnnhXOPPPd1e8nnXRi+bPFi5dopGEkhvmisliI%0AYcFmw1UeshYuvLy8H2Htu99dpnEBBskcLKpivaQYIirE/J0QZ5fFHKGobEXn%0AG516X3Ox4vfFWXD50g7x8wce+Fk5wfrxxx+peV6xtECcwVhU0aIic+aZZzS9%0AnEAMl0WV7eGHH9njTLqYT3booYem0057ezr22DcMur0abW/xPjGp/C1vOXWP%0A9abiOWvXPla9HScNFPLb9e3Wl2IZghD7aKBLaUQou+SSC8v9H8s7nHfeuenZ%0AZzekn//8n1vervwYeNOb3tgw9A1kfw/02OrE8ZLvyyOPPKL6mKj83X77yp7X%0AvKcm+BbHhDNxYWzpqlQqG3u+T9cUY1u+ZECskRQToQtLl36zOgm66Cyuvfbq%0AXl/n8ssXlp1VdHQR3K666q/26MCKTrC339eL973iir9o2EHFayxb9v1yG/sz%0AmDPmml0KoZhongeBRssyNNJMQIjO/JRTTivvX3fdNYNaqyw+17x5Z5b3v/rV%0AL6Tf+Z3ja97j1ltv7jecxr447rj/VN5/+OH/U7PPBrO/Wzm28raub5t2HC/1%0Arz9t2tSaEw16OyYGu8YZu/3eiufSque3VW9/8Ij90vVv1Y3RXt957OV00U83%0AVW8f/5qJ6UfvOaiVp29SwaLqtttuL29HBSH3tre9tbwdnVlMfm92NfC+OtPo%0A1OfPP79maYGoxBxxxGHle+Xvu3HjpuqcofqQ1eg9Yl7RgQfu+gf33nsfqHmP%0A6FhnzZrZckfXaHuLMBAee+zxshoTj4kOeN26f+vImXkPPbSq5n5UigYjqkwR%0AAIrPtnr1mmr75GeSRsWpv4CVV+MiqOf7ql37u5ljq9VjcjDHS3zm/PXyJTfy%0A14rvEcIivApZMPoJWFQ7vmJIIzqH+oUZo/MthglDLDHQTLUk/sovOp5dQ1hn%0Apde//sg0derUahUhn6AdHdxFF/1ZTQcelbJ4jZh4HY+LbYzOMa+gxfBQ3rlF%0AteF97/uDPTrl+nWiWl12Ira3PhxE5aJ+2CeGjq6//otle8aw2+zZs6rtFcOG%0AUQUqwkoxJBfDczEU1Yr6xUhbnXvVSISCoi2feuqp6vf4fMXPYuiwv7AYQ3WN%0Agnq79nczx1ZfOnG8FK/XW4Wq/piIkNVMNRAY2UxyJ/3gB/9Q3o6zBhtVDGIp%0AgUK+lENfoqMsgsiyZTdVh1liTld0LDF/qOhworONSkWjDieCSZwZVywXEZ1Z%0AdFiFfB5SVEziPRptf3R6MWRXyOfINGPJkm/UhKvoIBsthRCfYfHiG8qqVohK%0AVgzpRdUvfh9fEbp2VzxmlT9vttPNK0X5WYCDcdxxx+4RGiJsF20fnz8u7dNX%0ACM0v+ZMHjXbt72aOrb506ngpzuBsFMJim+rP1vzLv/xv/uEBAYvRLDrFqEwU%0A8rMGc1F9KUQlq6+OthAdclQIGlW7ig4yRCWjr8m/uyZh7+7sVqz4+z2CQJg3%0A77Q+tycmI7ejjaJT768jj/lDRUgI+cKi7VYMbQ3WtGnT9vhZ7Jd8qY6f/ez/%0ANRX68ue0c383c2z1pVPHS39ncMbnjc/d6v9DgIDFCBWdYr6OUm/XpIvqS/4X%0AeJz91Z94vRh+qRcdSz5U1EzVpjirMUSVJEJPiEpS8XX88cf1+Rrr168fUBut%0AWvVwTfUq35a+OtSPf/y88n4+x22kyYcv86BZLx8ezJ/Tzv3d37HVn04dL80c%0AE/G58/+H1qxZ6x8gELAYrfIhkzwQNFKsjdVbp1evt+HGp59e3/A1+wss+bDb%0AE088WXZaxVdvVZHYzmJOzUDk2xvb0Oyp9ieeeEJ5u9UhyeEk2jbmOYXehgnz%0A4cF4bB6i2rm/+zu2mvks7T5eWjkm8s8fy0MAo5dJ7mNYzAvKh0ziWnR9XSol%0AzoirD2d9DdHk84xy+VBSzE9qZumCerEGUaNKSEzYj7WbNm/eXO3A8jP7Birf%0A3voTAPrrzHMxl6hdE5tjvlSx7+J7X0tnDORz1s/riknkRTvGMGF9pTN/bj5f%0Ar537u5ljq1XtOF5aOSbyYcd27TdAwGKYiUURc/lCo82Iye59BayBznkaSFCM%0A1cdj+Kp+CYX6SsNATusfjurnSxWT6AcjwkVh7tw5Nb9797vPKNeNanQ2YT48%0AmM/X65TBHFtj8XgBBCyG0PLltw7q+fGX/mCrMvk6SK0FjF2n49efTh+i+hIB%0A4Zhjjq4+bsaMGdVtjG0dLR1m/bpXsS7WYNZWitCRV26i7XLFHLwY6iyGCYsq%0AVj48WFzDslP7e7DG6vECCFgMkegg8w61WJ+pGRdc8Kmyg2pm8cl6+fBWDCcN%0AdAXyCAV5ZxkVh1YWQW3W9Om7z9KrHybtb/tyU6ZMads21a9N1uq6XvXqq5mN%0AqlAf/OD7y7lk+TBhPgQYj+nU/s7nCw6n46WVYyIf6szPMgVGH5Pcx6j8LMBY%0ADyif/NvfVz4ZPoZZ+pvsvmc1Yvfw1qOPrh7wZ4ilD4rOMsJGzGdpd7gKeTWn%0AmbMnC/WrrbdjMdBcPtcpgk+j9aKaEfsvv5B3BI9G7ZjPNcrPJsyHBxvNR2rX%0A/h6sTh0vrRwTeRgrVnsHBCxGifhLvhjSCfPmndbS8/NL50SHVX+x4v7k1ZG4%0AQG+zAe1LX/qb6vXo4is+Q18TqxsZ6DIN+XBcMSzajPpFLdstlgbIqyCxeGWr%0AYTfEKuP5kNm55/5Rw8fFmXKx6nyx36MKmg8P1l8ap937e7A6dby0ckzkYayV%0AyfGAgMUIkC962dfaV70phqcKMTzVinxNreiomxn6iQ4sJuHHUFNc363VykN0%0A6jfcsHhA7VX/eSOQ9CfCRz5/p/76ju0QYeZzn7u6pqP/9KcvbSlkRYjJw3Ys%0A3tnXkG++EG0ME+ahpbegvjf292C1erx84xv/q9/HxBm6+bD8UJwMAAhYDKH8%0AUjf9rX3Vm/rhqTjdvRXnn/+x8nactt/X8hBRvchDTbHKdz43Kj5Tb8Eifh7B%0AYzDLNVx22cU1n/eaaz7b6/tFuLr00oXl/QgXnbq4b4TjoqpUbFt81v4qKtGm%0A8RnyM0djO/tbvDPerwibMUxYDA/2F9Tbsb8Hq5PHSwTBCKu9vWZMrs+Xp4hq%0A31CHRmBomeQ+xkTHm3cc+XBfK+r/+r777p+2NMeoCAZFBx+dT3R6EdyKU/C7%0Au7urq13H/KB87kyxanZUhYrqS3ym+fP/uOb5McSzevWa8nT8CAH5UFh0eocf%0AfnhT212/vfG+MdwVAbVYUDTeL6ozeeUq3nPhwss7uk+LJROKbYuQFV8RmGJh%0Ay3xJg5hkHfOg8qpVEa7iennNLJhZrIkVbVm8Tn9BvR37e7A6dbwUj4vPFkOA%0AjV4zD7Lx+PyyOQzMqzsr5e1pE9UKaL+D9h3X8HgTsGgozvrLO9WBTrzOT9sP%0A0SnGhXMHEwyi03vwwd4XoYyOadGia8sQEFWhfK2ivp5fXIx3/vzzy05zwYIL%0Aq89vdrHH+u2N1ynWhurrPds9ub23bYvFN+Oaf8XnK4JWf2JYMCpXza5Gnq+J%0A1UpQH+z+HqxOHS8RLu+++55qWzfzmeJ12/WZxrJD9huX1mzadXvlr19Jz72y%0AU6PQVk91b6853gQsehXDF/lf0s1etqQ3+Wn70QnFX/gDCQYxr+ezn72u1yGZ%0A6JTi0ijnnXfuHh1TXFQ5lgHoK+hE5eQDH3hfNRRGZ9jXY5vd3htv/Ntew0tf%0A29tJsfxBVHuiihbVob6GuIptLNplMOE6qkzNhsjB7u/B6sTxEmdJLl58Q3XY%0AMw+4jV53qI+J0eyM1+6bfvr01t90hDt6vl7WKHT0eGtVV6VS2djzfbrmY2+L%0AeVy//OUv0+bN3eXPYqilmXW2IjzGRZnz697Fc+fOnbNHhxZzpIrHxVDnQOfC%0AxFyhWIqhfnsbvefeas9nnnmmpk3yhTRH8v5uxx8bgzleIkwVc6quu+6amrW9%0AYhg+X+9q5swZ1QtLC1bttW1nSot+vjl98eHutF3xig6HqxvfeWBqcSR6k4AF%0A0KK+AhZDZ+uOSvr6L7ak2596Jb24taJBaKsJPYHqnbP3SfOPmpxmTh7f6tM3%0AGSIEYESaNL4rLXjDlOoXDDdOvQAAaLOWKlixonIhJtN2am2fmCydL0bY7Fle%0AuZjf8eUvf6W8/4lP/OmQnM0FANBSwMrX9+nkZR5i0mv+XgMJWC+99FLNa/R2%0A+Q8AgHYzRAgA0GYmuQO0KM4adOYg0BcVLAAAAQsAQMACABhTOjYHKy4XERcW%0AjqvL59cci4ulxhmIg7lECQDAmApYxfpT+RIJufh5fMUFXS+55FMmigIAAlZ/%0A4Wr+/PP3uJp8VK3CY489Xlaz4jFxLa916/4tffKTF9gTAICAVS+uTl8fruIi%0AqLHie34V+Rg6vP76L6Y77vjH6v1Fi76QZs+epZIFAIwabZvkvmTJN2rC1a23%0A3lwNTXm4Csce+4a0ePENZVUrRCVrw4YN9gYAMCq0pYIV1aubbvpeeT8qVxGk%0A+nLFFX+R7r33gTKU3XPPfcOmihVVtrPOOmfQrxMhciCX+QEARra2VLBWrXq4%0ApnoVw4L9icrWxz9+Xnn/tttutzcAgFGhLRWsp59eX96Oqk39sGBvTjzxhPJ2%0AMSdrODjkkIOrVbjBmjZtqiMMAASsgbnvvn8qb8caV82qH0aMobn+hhaHQqzP%0AZdI9ALBXAxZD4+1vPyP967+uS/vvPy3t3FnRIADQRjt37qzOKz/hhDemZcu+%0AJWCNFdu2bat+f/HFzRoDADpk69atg36NtgSs6dOnl7dj4dBm1S/NMGXKlF4f%0A1+plddavX9/Ua48kEydOrH5XwQKA9isqWJMmTRoeAeuYY44ub8e1B5tdmf2h%0Ah1bV3J8z53XV70ceecQej3vXu+a1tE2rV69p+NrNKC73M1jHHXdsmj//I23b%0A8T/5yUpHPwCMAG0JWG960xvL23EpnGYnq69ceUd5e8GCj5a3658bj2slYNWv%0Ay5W/djNeeumlXq+lCAAwJAErqkMnnHB8eZ3BuBROrNbel/vvf6AmxJx88kk1%0Av7/kkgurl9EJ8bg4O7HZM/uWLft+zbpc9a/dnxhOzFeaH6ioYAEAY09XpVLZ%0A2PN9ejMPPuyw3YEh1onKA08Epg9/ePfCoVE1uuiiP2u4JlY89tJLF5Yh6PTT%0A37FHIIt5V+9974f3uLZhfyFr6dJvpiuu2L16eqPXBgDooE1tC1jhS1/6m7Lq%0AFGbPnlldrb1YUDQmnsdwX165iscsXXpjwzlSd955V09Qu7DmZ1EpO/vss9Ks%0AWTPTjBkzqj/r7u5Oa9asTcuX31pW0fp7bQCAERGwGoWsvjQTgOqrXc2KILZo%0A0bXCFQAw5AFrXLtfMc4g/Pa3l1SH5voKVjHH6oc/XNFvADrllJOrj4tAF6Gp%0AmWAVj1269GvCFQCwV7RUwWpVzKOKJRY2b+4ufxZLMMydO6fp6xU2es1nn92Q%0A1q59rObnM2fOSIcdNrfl9bIAANpsU0cDFgDAWAxY47QBAEB7CVgAAAIWAICA%0ABQAgYAEAIGABAAhYAAACFgAAAhYAgIAFACBgAQAgYAEACFgAACPLBE0AMLat%0AePKVtPzxl9Ovu3ekri7tQXtUKj0hY1xKJx0yKX3oiP3Ssa+ZKGABMDY8/uL2%0A9HdPvJzuXPdqenVHRYPQdv/+ys60/8SuMRewuiqVysae79MdAgBjy/3PbE3n%0A/e9N6flXBSs6KEpZqZI+9+YD0pEHTEin/PaksfCpN6lgsec/uvc/kG65ZXl5%0A/9RTfzedffZ7NEyH2vquu36cvvrVr1fvz549M1155cL0rnfN6/e5d955V1q5%0A8o7y/rXXXt3y+z/55K/Sl7/8lfL+Jz7xp2nOnNe1/XPm23rGGac39fnovK//%0AYotwRed1VfNVWrJ6Szpq+pgJWIYI2VN0+Dff/Hfl/bgdneLkyZM1ThstX74i%0AXXzxZ2p+tm7d02nz5u6mnh+Py/fTQALWSy+9VPMa5577Rx35rPm2RmBneFj7%0AwnaNQOf9JsM/uXl7GjeG5vgJWNTYsGFDWU3J3XffP6k6tLmd83AVlas3v/lk%0ADcNe6fhgSA63ytg65gQsatxzz33l7XPO+f2y6vCd73xPwOpQO59wwvFp6dKv%0AqRAy5JwxyFCK6tVYOuasg0WNpUu/Vd6+9NI/r1ZWwh13/GN1vg7tN3/+H476%0AcBVz+B5//JHql/l8gIDFmPLII/+SHnxwVfX26ae/Ix188MHpQx96f/n7u+/+%0AqUYCAAGLVvz4xz8pb5955rur30866cTyZ4sXL9FIANAEc7Co2rJlS1q06Avl%0A/ThrMJxyysnVOUJR2Yoz3OJ0+77mYsXvi7Pg8qGg+PkDD/ws3XbbyurrxFBR%0ALqpnEfDiDMaiihbDk2eeeUY6+eSTmpr/FUOYUWV7+OFHas6MCzGf7NBDD02n%0Anfb2dOyxbxh0ezXa3uJ94iy5t7zl1GoFsP45a9c+Vr0dJw0U8tv17ba3Ndqf%0AcazEkgu33XZ7dei4/rM3OuM0/+xHHnlEn/tgIG3b7u0FGCwLjVKVLxmwYMFH%0A02c+c1n5u6VLv5muuOLqslPqazmAyy9fWIabCFHRuV111V/tEXiKgNXb7+vF%0A+15xxV807AjjNZYt+365jf256qqFaf78jwyonYp1o/rb3giHl1zyqZqw1GhZ%0Ahkbqw2cz+6yV59WHmbPOOqe8f+utN9eEn/r9GY+/4IJPVUNyb2J4+fOf/1zN%0Avsq39brrrmkYIgfTtu3e3rHijFufSw89v8M/gHTWb04fnDqxKx22/4S08qyD%0AxsKnttAou8Rf94WoGOXe9ra3lrej84rJ771VEPYMM72Hp+hQ588/v6bzi2rZ%0AEUccVr5X/r4bN25q2BE2eo/oNA88cNffDffe+0DNe0QQmzVrZstnRTba3iL8%0Ahccee7ysuMRjIlCsW/dv6ZOfvGBUHCPx+fOw0tu+iirRpz99aVq8+Ia92rad%0A3F6A/ghYVDuiYugkqgP1C0HGyt7FMGGIJQaaGcaKqkXRkcXzzz77rPT61x+Z%0Apk6dWq06XX31tWXnF4Hooov+rKZ6EpWyeI1Fi/66+rjYxghTeQUthoTyzjKq%0AU+973x/sEcLicVdeeXX5fq0uOxHbWx8AohJTP7wUFZPrr/9i2Z4x7Dp79qxq%0Ae8XQVlSIQgyBFUOyl1xyYXXocrgr9ldUOP/kT86rCdlRXVyy5BvlZ4rPH6vU%0AxxDzULTtUG4vQDNMcif94Af/UN6OswYbDZXEUgKFfCmHvkQwKjrLZctuqg7L%0ARQcWISrmxBQdZTFE02heTnSeS5feWC4XEWEqOtpCfqmY6EjjPRptf4SpGFbK%0AqxatiM44DwARlGLb6t8rPkNUQorKS4hqSywsGh18/D6+IhgU4nbx83bMD+uU%0AaLMIgzF8XF/BjHaIalL+uZ9+ev2Qte1Qbi+AgEVT1YObbvpeeT8/azAX1ZdC%0AVLLir/3+RKcZFaVGFYYifIWoXPU1/yUqaHk4WrHi78vbefVq3rzT+tyemFzd%0AjjaKwNhfEIoKSREKQ76w6EgVVcj+hjvf+96zy9v1k/eHum07sb0AAhZNiU6l%0AqB5Ep9XbEElUAKLSVIgzvPoTrxfDdfUinOVDg81UbYqzGkNcyic65qLaUXwd%0Af/xxfb7G+vUDq1CsWvVwTYUl35beRGD8+MfPK+/nc9xGqhji7U8M/w6Xtu3E%0A9gIIWDQlH2LLO61GirWx6kNOb3obbsyHYvLX7K9TzYdznnjiyer3fGittypY%0AbGcxB2sg8u2NbWj2bLMTTzyhvN3qkORwlH+edulk23ZiewGaZZL7GBZzV/Ih%0Athdf3FydVN6bOGurPpz1Ndk9n2eUy4diYg5NM0sX1Is1lRpVvmLC/s9//s9p%0A8+bN1fWw8rPPBirf3voTAPpSv30xd2w4z7HaG7QtIGAx6tx++8qa+/lCo82I%0Aye59BayBznkaSFD87neXVefy9LXeUX7xagAQsOiI5ctvHdTzozI02MpBvjZR%0AK6ZN2zV3pn75hRDzuubOnZOOOebo6uNmzJhR3cbYVgELAAGLjomJ5vnQWbE+%0AUzPyxRtjPadWA9Zxxx1bBp1Y/mGgl4aJylUerqJC1coiqM2aPn33hQ7qh0n7%0A277clClTOrpPi6UgWlE/8b/T2zhS2xagVSa5j1H5WYCxflQ+Wby/r3wyfAzL%0A9TfZvd60adPK248+unrAnyFOz89X6Y4FSNsdrkJUwhq1W38eeqh27lcsN9FO%0A9UOw9e/XjNWr13R0G0dq2wIIWLQs/vqPswAL8+ad1tLz80vnRMBpdf2gfE2t%0AuPhzswHtS1/6m+q15uIrPkP+vvlCqL0Z6DINb3rTG8vbxbBoM+oXQW23+sph%0A/n7NqF+DqhPbOFLbFkDAomX5wox9rX3Vm+LSOYW47Ewr8jW1IqA1Ewyi441J%0A+DG0GNcWbLVSFWHihhsWD6i96j9vXK6lPzEEm8/3qr++Y7vESuWFeL++zgKt%0AFxfIzueudWobR2rbAghYtCS/1E1/a1/1Jq8YxTpEsTxCK84//2Pl7Vimoa9g%0AENWqvOMtVnXP5+/EZ+qtEhY/j4v5Dma5hssuu7jm815zzWd7fb8IAJdeurC8%0AH2Gy1QtLN+sDH3hfzarm/bXl7vb6ZvWi10OxjSO1bQEGwyT3MSYqQXnQyIf7%0AWpEP84W77/5pS/NgomoW1ZdiaYgIBhGSIrgVc4u6u7vTmjVr0+LFS2rmWhWr%0AfUflohjqjM80f/4f1zw/hgRjjlGxfEMEkbxiE2cgHn744U1td/32xvvG8GYE%0A1GJBy3i/qMbl1ZV4z4ULL+/Y/oxK3pVXLkwLFlxYE7J2LaFxVpo1a2b1LMq8%0APePs0fwY6PQ2jtS2BRCwaFqc9Zf/9T/QycHFMF+xinaEoLjQciuK68QVHWt0%0A+g8+2Puio9GhLlp0bbnad1Qu8rWt+np+PDcuGj1//vllyIpQEs+PyfED2d54%0AnbwK1Nt7dnoCdrTDt7+9pFrZKT7brrbov2IXgTXadG9PEh+ubQswUIYIx5AY%0AdskXE232MjW9+eAH31/ejg4xKkID6VgjHOTzcBp1plHh+OEPV+zRocaFf+OC%0A0n2J595yy7erzx3okGj99ubXZWxlezslqkDxfnGx5L7aMg9W8dilS782bELK%0AcG1bgIHoqlQqG3u+T9cU7G0xj+uXv/xl2ry5u/xZDPc1s85WhMe4cHB+bbt4%0A7ty5c/a4vl3M4ykeF0OdA13aIeaGxXIB9dvb6D2HWmzbs89uqF5SKDdz5ox0%0A2GFzO7KcRbu3f7i27Whyxq3PpYee36Eh6KxKJf6Tpk7sSoftPyGtPOugsfCp%0ANwlYAAIWCFhtDliGCAEA2kzAAgAQsAAABCwAAAELAAABCwBAwAIAELAAABCw%0AAAAELAAAAQsAAAELAKDDJmgCAKDTtmyvpNWbtqdTb9kwMgJSV0rvnL1P+tjR%0Ak6sXqRawAIBhZ2clpa07KumJF7ePmG1e+8L29K21W9KPzjoozW0xZBkiBADo%0ARfe2Svr6L7a0/DwVLACgc7q6qt/G9Xzfd3xKR02fOCI2+/lXdqZfdW9PqVJJ%0A967fKmABAMMuZVWHCLf05JUHn9s2ora7J2GlrbHxLTJECADQh4m/qcIJWAAA%0Ae5GABQAgYAEACFgAAAIWAAACFgCAgAUAIGABACBgAQAIWAAAAhYAAAIWAICA%0ABQAgYAEACFgAAAhYAAACFgCAgAUAgIAFACBgAQAIWAAACFgAAAIWAICABQCA%0AgAUAIGABAAhYAAACFgAAAhYAgIAFACBgAQAgYAEACFgAAAIWAAACFgCAgAUA%0AIGABACBgAQAIWAAAAhYAgIAFAICABQAgYAEACFgAAAhYAAACFgCAgAUAgIAF%0AACBgAQAIWAAACFgAAAIWAICABQAgYAEAIGABAAhYAAACFgAAAhYAgIAFACBg%0AAQAgYAEACFgAAAIWAICABQCAgAUAIGABAAhYAAAIWAAAAhYAgIAFAICABQAg%0AYAEACFgAAAhYAAACFgCAgAUAIGABACBgAQAIWAAAAhYAAAIWAICABQAgYAEA%0AIGABAAhYAAACFgAAAhYAgIAFACBgAQAIWAAACFgAAAIWAICABQCAgAUAIGAB%0AAAhYAAAIWAAAAhYAgIAFAICABQAgYAEACFgAAAIWAAACFgAtq1S0AQhYALRX%0AlyYAAQuAtjp8/wkaATrE/10AY9RHj5qcHt24Lf1i046R/UG6lOIYflSwAMao%0AU2dMSv/znQem0//DpDRhXEzIGqlfMPyoYAGMYUceMCF94IjJadK4rvTr7h0j%0Aphi09oXt6eXtu8JVl5iFgAXAcPOeOftWv0aS31vxXFr1/Lbq7fHjUtq+035k%0AeDFECAAgYAEACFgAAAIWAAACFgCAgAUAIGABACBgAQAIWAAAAhYAAAIWAICA%0ABQAgYAEACFgAAAhYAAACFgCAgAUAgIAFACBgAQAIWAAACFgAAAIWAICABQCA%0AgAUAIGABAAhYAAACFgAAAhYAgIAFACBgAQAgYAEACFgAAAIWAAACFgCAgAUA%0AIGABACBgATBCvbqzohEY1sebgAXAiPPb+40vb2/fqT0YuuNNwAJg1Hr3ofto%0ABIZAZcDH2wSNB8BI89GjpqSNr1bS9Q91p207lbDonP/8un2rx5uABcCoN2Fc%0ASp86fko6YFJXuv2pV9KLW83Jov3H2Dtn75PmHzW5ertVXZVKZWPP9+maEgCg%0ALTaZgwUA0GYCFgCAgAUAIGABAAhYAAAIWAAAAhYAgIAFAICABQAgYAEAjAJx%0AqZxtyTUJAQDaZXsEq3U9XwdoCwCAtnjh/wswAENxPKI47odIAAAAAElFTkSu%0AQmCC)

基于此提出IoU Loss,其将4个点构成的box看成一个整体进行回归

![Images](...%0AIiICIiAiIgIiICIiD//Z)

上图中的红色点表示目标检测网络结构中Head部分上的点（i,j），绿色的框表示Ground truth框, 蓝色的框表示Prediction的框，IoU loss的定义如上，先求出2个框的IoU，然后再求个-ln(IoU)，实际很多是直接定义为IoU Loss = 1-IoU

# 3.GIOU(Generalized IOU)-目标检测任务的新Loss
对于两个任意凸型(体积)$A,B \subseteq S \in R^n$，首先求出了包含A和B的最小凸型$C \subseteq \in R^n$，为了比较两种特定的几何形状，C可以来自同一类型

我们找到一个最小的封闭形状C，让C可以把A，B包含在内，然后我们计算C中没有覆盖A和B的面积占C总面积的比值，然后用A与B的IoU减去这个比值

![image](...%0AClKUClKUClKUClKUClKUClKUClKUClKUH//Z)

## 2.1.GIOU的性质
1. 与IOU类似，GIOU也可以作为一个距离，loss可以用下面的公式计算$L_{GIOU}=1-L_{GIOU}$
2. 同原始IoU类似，GIoU对物体的大小不敏感。GIoU总是小于等于IoU，对于IoU，有:$0 \leq IOU \leq 1$，而GIOU则是$-1 \leq GIOU \leq 1$
3. 由于GIoU引入了包含A，B两个形状的C，所以当A，B不重合时，依然可以进行优化
4. 与IoU相比，GIoU不仅关注重叠区域。封闭形状C中两个对称形状A和B之间的空隙在A和B之间没有很好的对齐时增大,因此GIoU的值更能反映两个对称物体之间是如何重叠的

总之就是保留了IoU的原始性质同时弱化了它的缺点

## 2.2.GIOU作为BBox回归的损失
假设有输入为预测框$B^p=(x)1^p,y_1^p,x_2^p,y_2^p)$和真实框$B_g=(x_1^g,y_1^g,x_2^g,y_2^g)$。输出为$L_{IOU}$和$L_{GIOU}$
- 对于预测框$B^p$来说，有
$$x_2^p > x_1^p, y_2^p > y_1^p$$
且有
$$
\begin{split}
\hat{x}_1^p&=min(x_1^p,x_2^p) \\
\hat{x}_2^p&=max(x_1^p,x_2^p) \\
\hat{y}_1^p&=min(y_1^p,y_2^p) \\
\hat{y}_2^p&=max(y_1^p,y_2^p)
\end{split}
$$
- 计算$B^g$的面积
$$A^g=(x_2^g-x_1^g) \times (y_2^g-y_1^g)$$
- 计算$B^p$的面积
$$A^p=(x_2^p-x_1^p) \times (y_2^p-y_1^p)$$
- 计算$B^g,B^p$的重叠面积
假设如下
$$
\begin{split}
x_1^I&=max(\hat{x}_1^p,x_1^g) \\
x_2^I&=min(\hat{x}_2^p,x_2^g) \\
y_1^I&=max(\hat{y}_1^p,y_1^g) \\
y_2^I&=max(\hat{y}_2^p,y_2^g)
\end{split}
$$
有
$$
I=\begin{cases}
(x_2^I-x_1^I) \times (y_2^I-y_1^I) & x_2^I > x_1^I,y_2^i > y_1^I \\
0 & otherwise
\end{cases}
$$
- 找到可以包含$B^p,B^g$的最小的box $B^c$
$$
\begin{split}
x_1^c&=min(\hat{x}_1^p,x_1^g) \\
x_2^c&=max(\hat{x}_2^p,x_2^g) \\
y_1^c&=min(\hat{y}_1^p,y_1^g) \\
y_2^c&=max(\hat{y}_2^p,y_2^g)
\end{split}
$$
- 计算$B^c$的面积
$$A^c=(x_2^c-x_1^c) \times (y_2^c-y_1^c)$$
- 计算IOU
$$IOU=\frac{I}{U}=\frac{I}{A^p+A^g-I}$$
- 计算GIOU
$$GIOU=IOU-\frac{A^c-U}{A^c}$$
- 计算损失
$$L_{GIOU}=1-GIOU$$

## 2.3.计算GIOU

In [1]:
import numpy as np
def bbox_overlaps_giou(bboxes1, bboxes2):
    """Calculate the gious between each bbox of bboxes1 and bboxes2.
    Args:
        bboxes1(ndarray): shape (n, 4)
        bboxes2(ndarray): shape (k, 4)
    Returns:
        gious(ndarray): shape (n, k)
    """


    bboxes1 = bboxes1.astype(np.float32)
    bboxes2 = bboxes2.astype(np.float32)
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    ious = np.zeros((rows, cols), dtype=np.float32)
    if rows * cols == 0:
        return ious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        ious = np.zeros((cols, rows), dtype=np.float32)
        exchange = True
    area1 = (bboxes1[:, 2] - bboxes1[:, 0] + 1) * (
        bboxes1[:, 3] - bboxes1[:, 1] + 1)
    area2 = (bboxes2[:, 2] - bboxes2[:, 0] + 1) * (
        bboxes2[:, 3] - bboxes2[:, 1] + 1)
    for i in range(bboxes1.shape[0]):
        x_start = np.maximum(bboxes1[i, 0], bboxes2[:, 0])
        x_min = np.minimum(bboxes1[i, 0], bboxes2[:, 0])
        y_start = np.maximum(bboxes1[i, 1], bboxes2[:, 1])
        y_min = np.minimum(bboxes1[i, 1], bboxes2[:, 1])
        x_end = np.minimum(bboxes1[i, 2], bboxes2[:, 2])
        x_max = np.maximum(bboxes1[i, 2], bboxes2[:, 2])
        y_end = np.minimum(bboxes1[i, 3], bboxes2[:, 3])
        y_max = np.maximum(bboxes1[i, 3], bboxes2[:, 3])

        overlap = np.maximum(x_end - x_start + 1, 0) * np.maximum(y_end - y_start + 1, 0)
        closure = np.maximum(x_max - x_min + 1, 0) * np.maximum(y_max - y_min + 1, 0)

        union = area1[i] + area2 - overlap
        closure
        ious[i, :] = overlap / union - (closure - union) / closure
    if exchange:
        ious = ious.T
    return ious

In [2]:
box1 = np.array([[0,0,100,100]]*3)
box2 = np.array([[0,200,100,300]]*3)
print(bbox_overlaps_giou(box1, box2))

[[-0.32890365 -0.32890365 -0.32890365]
 [-0.32890365 -0.32890365 -0.32890365]
 [-0.32890365 -0.32890365 -0.32890365]]


# 3.DIOU：更加稳定有效的目标框回归损失

## 3.1.IOU回顾
$$
\begin{cases}
IOU&=\frac{\left|B \cap B^{gt} \right|}{\left|B \cup B^{gt} \right|} \\
\mathcal{L}_{IOU}&=1-IOU=1-\frac{\left|B \cap B^{gt} \right|}{\left|B \cup B^{gt} \right|}
\end{cases}
$$

IoU的缺点是当两个框不想相交时，IoU损失总是1，不能给出优化方向。

## 3.2.GIOU回顾
$$
\begin{cases}
GIOU&=IOU-\frac{\left|C-B \cup B^{gt} \right|}{\left| C \right|} \\
\mathcal{L}_{GIOU}&=1-IOU+\frac{\left|C-B \cup B^{gt} \right|}{\left| C \right|}
\end{cases}
$$

GIOU在IOU的基础上添加了一个项，其中C表示包含两个框的最小矩形，因此可以优化两个框不相交的情况。不过，GIOU也存在着问题。当两个框相交时，GIOU损失退化为IOU损失。因此，当包含预测框bbox和地面真值bbox时，特别是在水平和垂直方向上，很难进行优化。这里的一个猜想是，在水平方向和垂直方向上，损失的值没有在其他方向上增长得快，因此对这两个方向的惩罚不够，导致收敛速度减慢

![Images](data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsL%0ADBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/%0A2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy%0AMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCADVAcQDASIAAhEBAxEB/8QA%0AHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUF%0ABAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkK%0AFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1%0Adnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXG%0Ax8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEB%0AAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAEC%0AAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRom%0AJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOE%0AhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU%0A1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD34nCk+grn%0AJPFiRyMn2ZjtJHWuif7jfQ1xmiQRXGuSpNGsi/OcMMjrXDi6lSMoRpu1zKpK%0ASaS6l/8A4S5P+fZvzo/4S5P+fZvzra/srT/+fOH/AL4FH9laf/z5w/8AfApe%0Ayxf86+4XLU7mL/wlyf8APs350f8ACXJ/z7N+dbX9laf/AM+cP/fAo/srT/8A%0Anzh/74FHssX/ADr7g5ancxf+EuT/AJ9m/Oj/AIS5P+fZvzra/srT/wDnzh/7%0A4FH9laf/AM+cP/fAo9li/wCdfcHLU7mL/wAJcn/Ps350f8Jcn/Ps351tf2Vp%0A/wDz5w/98Cj+ytP/AOfOH/vgUeyxf86+4OWp3MX/AIS5P+fZvzo/4S5P+fZv%0Azra/srT/APnzh/74FH9laf8A8+cP/fAo9li/519wctTuYv8Awlyf8+zfnR/w%0Alyf8+zfnW1/ZWn/8+cP/AHwKP7K0/wD584f++BR7LF/zr7g5ancxf+EuT/n2%0Ab86v6Vri6nO8QhKbV3ZJqxNpdgIJCLSEEKf4B6VgeFRjU7gD+4f51HNiKdaE%0AZyumK84ySbOuooor0jcKKKKAGTSeVC8mM7RnFc1/wlw/59f1rorv/jzm/wBw%0A1ynhe2huLicTRJIAoxuGcda4MVUqqrCnTdr3MajlzJRLP/CXD/n1/Wj/AIS4%0Af8+v61u/2ZY/8+kP/fIo/syx/wCfSH/vkUexxf8Az8X3By1O5hf8JcP+fX9a%0AP+EuH/Pr+tbv9mWP/PpD/wB8ij+zLH/n0h/75FHscX/z8X3By1O5hf8ACXD/%0AAJ9f1o/4S4f8+v61u/2ZY/8APpD/AN8ij+zLH/n0h/75FHscX/z8X3By1O5h%0Af8JcP+fX9aP+EuH/AD6/rW7/AGZY/wDPpD/3yKP7Msf+fSH/AL5FHscX/wA/%0AF9wctTuYX/CXD/n1/Wj/AIS4f8+v61u/2ZY/8+kP/fIo/syx/wCfSH/vkUex%0Axf8Az8X3By1O5hf8JcP+fX9aP+EuH/Pr+tbv9mWP/PpD/wB8ij+zLH/n0h/7%0A5FHscX/z8X3By1O5hf8ACXD/AJ9f1q/pWvf2ndtB5OzCFs59x/jVyTTLHy2/%0A0SHof4RXN+GQBrkwAwBG3/oQrNyxFKrCM5XTFecZJN7nY0UUV6ZuFFFFACMd%0Aqlj2Gawj4rswSPKk4rbl/wBS/wDumuL0Gygvb+VJ03KASBmuLFVasZwhT6mV%0ASUk0o9TY/wCErs/+eUtH/CV2f/PKWrn/AAj+m/8APuPzo/4R/Tf+fcfnU8uM%0A/mQrVe6Kf/CV2f8Azylo/wCErs/+eUtXP+Ef03/n3H50f8I/pv8Az7j86OXG%0AfzILVe6Kf/CV2f8Azylo/wCErs/+eUtXP+Ef03/n3H50f8I/pv8Az7j86OXG%0AfzILVe6Kf/CV2f8Azylo/wCErs/+eUtXP+Ef03/n3H50f8I/pv8Az7j86OXG%0AfzILVe6Kf/CV2f8Azylo/wCErs/+eUtXP+Ef03/n3H50f8I/pv8Az7j86OXG%0AfzILVe6Kf/CV2f8Azylo/wCErs/+eUtXP+Ef03/n3H50f8I/pv8Az7j86OXG%0AfzILVe6Kf/CV2f8Azylq3p+uW+o3JgiRwwUtk0j+H9NCMRB29awvDShdclUd%0AAjAfmKj2mJp1IRqNWYuacZJPqdlRRRXpG41/uN9DXIeHv+Rgl+j/AM669/uN%0A9DXIeHv+Rgl+j/zrgxX8al6mNT4onY0UUV3mwUUUUAFFFFABRRRQAUUUUAFF%0AFFAEc/8Ax7yf7h/lXK+Fv+Qncf7h/nXVT/8AHvJ/uH+Vcr4W/wCQncf7h/nX%0ABif94pfMxn8cTrqKKK7zYKKKKAIbv/jzm/3DXMeEf+Pm5/3R/Wunu/8Ajzm/%0A3DXMeEf+Pm5/3R/WuCv/AL1S+ZjP+JE62iiiu82CiiigAooooAKKKKACiiig%0AAooooAbJ/q3+hrkfDX/Idn/65t/6EK66T/Vv9DXI+Gv+Q7P/ANc2/wDQhXBi%0Av49L1ManxxOwooorvNgooooAZL/qX/3TXJ+Fv+QnN/u11kv+pf8A3TXJ+Fv+%0AQnN/u1wYn+PS+ZjU+OJ19FFFd5sFFFFABRRRQAUUUUAFFFFABRRRQA1/9W30%0ANch4b/5D03+4/wDMV17/AOrb6GuQ8N/8h6b/AHH/AJiuDFfxqXqY1PiidjRR%0ARXebDX+430Nch4e/5GCX6P8Azrr3+430Nch4e/5GCX6P/OuDFfxqXqY1Piid%0AjRRRXebBRRRQAUUUUAFFFFABRRRQAUUUUARz/wDHvJ/uH+Vcr4W/5Cdx/uH+%0AddVP/wAe8n+4f5Vyvhb/AJCdx/uH+dcGJ/3il8zGfxxOuooorvNgooooAhu/%0A+POb/cNcx4R/4+bn/dH9a6e7/wCPOb/cNcx4R/4+bn/dH9a4K/8AvVL5mM/4%0AkTraKKK7zYKKKKACiiobpilpMynDBCQfwoGld2JqK+fdP8R+NtXupYNPv72d%0A0G4qj9BnFaW74mf3tR/76H+NcSxqeqiz6WfDM6b5Z14J+bPcKK8P3fEz+9qP%0A/fQ/xo3fEz+9qP8A30P8af1v+4yf9Xf+oin957hRXh+74mf3tR/76H+NG74m%0Af3tR/wC+h/jR9b/uMP8AV3/qIp/ee3Sf6t/oa5Hw1/yHZ/8Arm3/AKEK89d/%0AiUsbM7aiFAJPzDpV34V6pqF54vuIru4kkAtHJVj33rXPUq+1rU9GrPqYYnhy%0ApClKvGrGShq7O57LRRRXqHhBRRRQAyX/AFL/AO6a5Pwt/wAhOb/drrJf9S/+%0A6a5Pwt/yE5v92uDE/wAel8zGp8cTr6KKK7zYKK80+K+s6lpP9n/YL2a237t3%0AltjNcrbW/wASLy1iuYJr94ZUDo3nqMgjIPWuWeK5ZuCi3Y97C5E62HjiJVYw%0AUu+h7rRXiH9nfE3/AJ6X/wD4EL/jR/Z3xN/56X//AIEL/jU/W3/IzX/V+H/Q%0ATT+89vorxD+zvib/AM9L/wD8CF/xo/s74m/89L//AMCF/wAaPrb/AJGH+r8P%0A+gmn957fRXiH9nfE3/npf/8AgQv+NZer6j440Ixf2lfX0Hm52ZmBzj6Gk8Zy%0Aq7gy6fDftJclPEQb7J3PoOisrw3PLc+G7CaZ2kleEFmY8k1q12Rd1c+cqQdO%0Abg+jsNf/AFbfQ1yHhv8A5D03+4/8xXXv/q2+hrkPDf8AyHpv9x/5iuHFfxqX%0Aqc1T4onY0UUV3mw1/uN9DXIeHv8AkYJfo/8AOuvf7jfQ1yHh7/kYJfo/864M%0AV/GpepjU+KJ2NFFFd5sFFFFABRRRQAUUUUAFFFFABRRRQBHP/wAe8n+4f5Vy%0Avhb/AJCdx/uH+ddVP/x7yf7h/lXK+Fv+Qncf7h/nXBif94pfMxn8cTrqKKK7%0AzYKKKKAIbv8A485v9w1zHhH/AI+bn/dH9a6e7/485v8AcNcx4R/4+bn/AHR/%0AWuCv/vVL5mM/4kTraKKK7zYKKKKACoL3/jyn/wCubfyqeoL3/jyn/wCubfyp%0APYqHxI8e+Dv/ACM2pf8AXt/7OK9orxf4O/8AIzal/wBe3/s4r2iuXBfwUe/x%0AP/yMZei/IKKKK6z54KKKKAIbz/jyn/65t/KvGfhV/wAj9ff9e0v/AKMWvZrz%0A/jyn/wCubfyrxn4Vf8j9ff8AXtL/AOjFrixH8amfR5R/uGL/AMK/U9sooort%0APnAooooAZL/qX/3TXJ+Fv+QnN/u11kv+pf8A3TXJ+Fv+QnN/u1wYn+PS+ZjU%0A+OJ19FFFd5seTfGfrpn/AAKvQ/C//IqaT/16Rf8AoIrzz4z9dM/4FXofhf8A%0A5FTSf+vSL/0EVx0v94n8j6PHf8ibDesv1Naiiiuw+cCiiigAryf40fd0v6vX%0ArFeT/Gj7ul/V65sZ/BZ7nDn/ACMqfz/JnoHhT/kVdN/64Ctisfwp/wAirpv/%0AAFwFbFbU/gR5eL/3ifq/zGv/AKtvoa5Dw3/yHpv9x/5iuvf/AFbfQ1yHhv8A%0A5D03+4/8xXHiv41L1OGp8UTsaKKK7zYa/wBxvoa5Dw9/yMEv0f8AnXXv9xvo%0Aa5Dw9/yMEv0f+dcGK/jUvUxqfFE7Giiiu82CiiigAooooAKK5s+PvC6kg6vE%0ACOvyP/hR/wAJ/wCFv+gxF/3w/wDhUOrBbtHUsDinqqUvuf8AkdJRXN/8J/4W%0A/wCgxF/3w/8AhR/wn/hb/oMRf98P/hS9rT/mX3lf2fi/+fUv/AX/AJHSUVzf%0A/Cf+Fv8AoMRf98P/AIUf8J/4W/6DEX/fD/4Ue1p/zL7w/s/F/wDPqX/gL/yO%0Agn/495P9w/yrlfC3/ITuP9w/zqebx94XaGQDV4iSpA+R/wDCsvwVq1hf6vcx%0A2twsjCMtgA9Mj1rjryjKvT5Xfc562BxUGpypySXWzO7ooor0DMKKKKAIbv8A%0A485v9w1zHhH/AI+bn/dH9a6e7/485v8AcNcx4R/4+bn/AHR/WuCv/vVL5mM/%0A4kTraKKK7zYKKKKACoL3/jyn/wCubfyqeoL3/jyn/wCubfypPYqHxI8e+Dv/%0AACM2pf8AXt/7OK9orxj4PAjxLqPH/Lt/7OK9nrlwX8FHvcTtPMZW7L8goooJ%0AA6nFdZ8+FFFFAEN5/wAeU/8A1zb+VeM/Cr/kfr7/AK9pf/Ri17Nef8eU/wD1%0Azb+VeM/Cr/kfr7/r2l/9GLXFiP41M+jyj/cMX/hX6ntlFFFdp84FFFFADJf9%0AS/8AumuT8Lf8hOb/AHa6yX/Uv/umuT8Lf8hOb/drgxP8el8zGp8cTr6KKK7z%0AY8m+M/XTP+BV6H4X/wCRU0n/AK9Iv/QRXnnxn66Z/wACr0Pwv/yKmk/9ekX/%0AAKCK46X+8T+R9Hjv+RNhvWX6mtRRRXYfOBRTQ6lioYFh1APIp1ABXk/xo+7p%0Af1evWK8n+NH3dL+r1zYz+Cz3OHP+RlT+f5M9A8Kf8irpv/XAVsVj+FP+RV03%0A/rgK2K2p/Ajy8X/vE/V/mNf/AFbfQ1yHhv8A5D03+4/8xXXv/q2+hrkPDf8A%0AyHpv9x/5iuPFfxqXqcNT4onY0UUV3mw1/uN9DXIeHv8AkYJfo/8AOuvf7jfQ%0A1yHh7/kYJfo/864MV/GpepjU+KJ2NFFFd5sFFFFABRRRQB88+C/Ddt4p125t%0ALmV4kVC+U6nmu/8A+FOaP/z+3P5Cua+Ef/I2Xv8A1xP869srz8LShUp801dn%0A1+e5jisHjHSw83GNlp8jzj/hTmj/APP7c/kKP+FOaP8A8/tz+Qr0eiuj6rR/%0AlPG/t3Mf+fr/AAPOP+FOaP8A8/tz+Qo/4U5o/wDz+3P5CvR6KPqtH+UP7dzH%0A/n6/wPMrv4Q6TBZTzLe3O6ONmGQOoGawfg+uzxTernOLZh/48tew6l/yC7v/%0AAK4v/wCgmvIPhF/yNt//ANe7f+hiuapShTrw5VY9fC4/E4vLcV7efNZK33nt%0ANFFFeifJBRRRQBDd/wDHnN/uGuY8I/8AHzc/7o/rXT3f/HnN/uGuY8I/8fNz%0A/uj+tcFf/eqXzMZ/xInW0UUV3mwUUUUAFQXv/HlP/wBc2/lU9QXv/HlP/wBc%0A2/lSexUPiR438I4xL4rvJN7Dy4CdvY5bH9a9rrxf4O/8jNqX/Xt/7OK9ormw%0AbbpXZ7nEkFDHuK7L8grk/HehavrmnQR6Td+S8cm5k3Fdw+o9K6yiumSUlZnj%0AUasqNRVI7rvqUdHtbiy0i1trqXzZ44wrv6mr1FFMzbu7kN5/x5T/APXNv5V4%0Az8Kv+R+vv+vaX/0YtezXn/HlP/1zb+VeM/Cr/kfr7/r2l/8ARi1xYj+NTPos%0Ao/3DF/4V+p7ZRRRXafOBRRRQAyX/AFL/AO6a5Pwt/wAhOb/drrJf9S/+6a5P%0Awt/yE5v92uDE/wAel8zGp8cTr6KKK7zY8m+M/XTP+BV6H4X/AORU0n/r0i/9%0ABFeefGfrpn/Aq9D8L/8AIqaT/wBekX/oIrjpf7xP5H0eO/5E2G9Zfqa1FFFd%0Ah84cT4e8Javpfiy61O61MTW0m7amTls9MjoMV21FFKMVHRGtatOtLmnvZLa2%0AwV5P8aPu6X9Xr1ivJ/jR93S/q9c+M/gs9fhz/kZU/n+TPQPCn/Iq6b/1wFbF%0AY/hT/kVdN/64CtitqfwI8vF/7xP1f5jX/wBW30Nch4b/AOQ9N/uP/MV17/6t%0Avoa5Dw3/AMh6b/cf+YrjxX8al6nDU+KJ2NFFFd5sNf7jfQ1yHh7/AJGCX6P/%0AADrr3+430Nch4e/5GCX6P/OuDFfxqXqY1PiidjRRRXebBRRRQAUUUUAeJ/CP%0A/kbL3/rif517ZXifwj/5Gy9/64n+de2Vx4H+CfRcUf8AIwfogooorsPnQooo%0AoAq6l/yC7v8A64v/AOgmvIPhF/yNt/8A9e7f+hivX9S/5Bd3/wBcX/8AQTXk%0AHwi/5G2//wCvdv8A0MVxYj+PTPosr/5FuL9I/me00UUV2nzoUUUUAQ3f/HnN%0A/uGuY8I/8fNz/uj+tdPd/wDHnN/uGuY8I/8AHzc/7o/rXBX/AN6pfMxn/Eid%0AbRRRXebBRRRQAVBe/wDHlP8A9c2/lU9QXv8Ax5T/APXNv5UnsVD4kePfB3/k%0AZtS/69v/AGcV7RXi/wAHf+Rm1L/r2/8AZxXtFcuC/go9/if/AJGMvRfkFFFF%0AdZ88FFFFAEN5/wAeU/8A1zb+VeM/Cr/kfr7/AK9pf/Ri17Nef8eU/wD1zb+V%0AeM/Cr/kfr7/r2l/9GLXFiP41M+jyj/cMX/hX6ntlFFFdp84FFFFADJf9S/8A%0AumuT8Lf8hOb/AHa6yX/Uv/umuT8Lf8hOb/drgxP8el8zGp8cTr6KKK7zY8m+%0AM/XTP+BV6H4X/wCRU0n/AK9Iv/QRXnnxn66Z/wACr0Pwv/yKmk/9ekX/AKCK%0A46X+8T+R9Hjv+RNhvWX6mtRRRXYfOBRRRQAV5P8AGj7ul/V69Yryf40fd0v6%0AvXNjP4LPc4c/5GVP5/kz0Dwp/wAirpv/AFwFbFY/hT/kVdN/64CtitqfwI8v%0AF/7xP1f5jX/1bfQ1yHhv/kPTf7j/AMxXXv8A6tvoa5Dw3/yHpv8Acf8AmK48%0AV/Gpepw1PiidjRRRXebDX+430Nch4e/5GCX6P/Ouvf7jfQ1yHh7/AJGCX6P/%0AADrgxX8al6mNT4onY0UUV3mwUUUUAFFFFAHifwj/AORsvf8Arif517ZXifwj%0A/wCRsvf+uJ/nXtlceB/gn0XFH/IwfogrA8U+K7PwraRT3SSSGVtqInf1rfqj%0Aqej6frEKw6hapcRqdwDdj+Fdcr203PCoumqidVNx623JdPvotS0+C9gz5UyB%0A13DnFWajggitoEhhQJGgwqjoBUlMzdr6FXUv+QXd/wDXF/8A0E15B8Iv+Rtv%0A/wDr3b/0MV6/qX/ILu/+uL/+gmvIPhF/yNt//wBe7f8AoYrixH8emfQ5X/yL%0AcX6R/M9pooortPnQooooAhu/+POb/cNcx4R/4+bn/dH9a6e7/wCPOb/cNcx4%0AR/4+bn/dH9a4K/8AvVL5mM/4kTraKKK7zYKKKKACoL3/AI8p/wDrm38qnpGU%0AOpVhlSMEUMcXZpnzv4L8VReE9Xu7qW2acSxmMKrYx8wP9K7n/hc9l/0Cpv8A%0Av6P8K7E+CfDZOTpEH6/40f8ACE+Gv+gRB+v+NcFOhiKceWMlY+rxea5Ti6nt%0Aa1GTl6/8E47/AIXPZf8AQKm/7+j/AAo/4XPZf9Aqb/v6P8K7H/hCfDX/AECI%0AP1/xo/4Qnw1/0CIP1/xq+TE/zI5vrWR/8+Jff/wTjv8Ahc9l/wBAqb/v6P8A%0ACj/hc9l/0Cpv+/o/wrsf+EJ8Nf8AQIg/X/Gj/hCfDX/QIg/X/GjkxP8AMg+t%0AZH/z4l9//BOKn+MlnLbyxjSpcshUfvR3H0rE+Eknm+N7uTGN9pI2PTLpXpz+%0ACfDYjYjSIOh9f8axPB+l2Nj4gne2tkjbynXIz03CsJqpGtT9o76jqZvltHD1%0AKGGpSi6itq7/AKneUUUV6Z82FFFFADJf9S/+6a5Pwt/yE5v92usl/wBS/wDu%0AmuT8Lf8AITm/3a4MT/HpfMxqfHE6+iiiu82PJvjP/wAwz/gVUdL+Lr6bpVpY%0A/wBjiT7PCsW/7RjdgYzjbXrt3p1lf7ftdrFPt+75iA4qt/wjmi/9Au0/79Cu%0AOdCp7RzhK1z6PD5tgvqcMNiaLny36239Dzb/AIXU/wD0Ax/4E/8A2NH/AAup%0A/wDoBj/wJ/8Asa9J/wCEc0X/AKBdp/36FH/COaL/ANAu0/79Cj2WI/n/AAD6%0A9k3/AECv/wACf+Z5t/wup/8AoBj/AMCf/saP+F1P/wBAMf8AgT/9jXpP/COa%0AL/0C7T/v0KP+Ec0X/oF2n/foUeyxH8/4B9eyb/oFf/gT/wAzzb/hdT/9AMf+%0ABP8A9jXKeM/GzeLhag2ItvIz0k35z+Ar3T/hHNF/6Bdp/wB+hR/wjmi/9Au0%0A/wC/QqZ4evOPLKenob4bN8rw1RVaWHakv7z/AFIvCn/Iq6b/ANcBWxTIoo4Y%0AljiQIijCqowAKfXbFWSR8zWn7SpKa6tsa/8Aq2+hrkPDf/Iem/3H/mK69/8A%0AVt9DXIeG/wDkPTf7j/zFcOK/jUvU5anxROxooorvNhr/AHG+hrkPD3/IwS/R%0A/wCdde/3G+hrkPD3/IwS/R/51wYr+NS9TGp8UTsaKKK7zYKKKKACiiigDxP4%0AR/8AI2Xv/XE/zr2yvE/hH/yNl7/1xP8AOvbK48D/AAT6Lij/AJGD9EFFFFdh%0A86FFFFAFXUv+QXd/9cX/APQTXkHwi/5G2/8A+vdv/QxXr+pf8gu7/wCuL/8A%0AoJryD4Rf8jbf/wDXu3/oYrixH8emfRZX/wAi3F+kfzPaaKKK7T50KKKKAIbv%0A/jzm/wBw1zHhH/j5uf8AdH9a6e7/AOPOb/cNcx4R/wCPm5/3R/WuCv8A71S+%0AZjP+JE62iiiu82CiiigAooooAKKKKACiiigAooooAbJ/q3+hrkfDX/Idn/65%0At/6EK66T/Vv9DXI+Gv8AkOz/APXNv/QhXBiv49L1ManxxOwooorvNgooooAZ%0AL/qX/wB01yfhb/kJzf7tdZL/AKl/901yfhb/AJCc3+7XBif49L5mNT44nX0U%0AUV3mwUUUUAFFFFABRRRQAUUUUAFFFFADX/1bfQ1yHhv/AJD03+4/8xXXv/q2%0A+hrkPDf/ACHpv9x/5iuDFfxqXqY1PiidjRRRXebDX+430Nch4fIGvy5PZ/51%0A2DDKkeorj38OamLh5I2jXLEgiTBxXBjFPnhOMb2ZjVTumkdhuX+8Pzo3L/eH%0A51yH9gaz/wA9x/3+NH9gaz/z3H/f40fW6v8Az6Ye0l/KdfuX+8Pzo3L/AHh+%0Adch/YGs/89x/3+NH9gaz/wA9x/3+NH1ur/z6Ye0l/KdfuX+8Pzo3L/eH51yH%0A9gaz/wA9x/3+NH9gaz/z3H/f40fW6v8Az6Ye0l/KcF8JXVfFd4WYAeSep969%0Aq86L/non/fQrw9PhZ4shlZ4ZLaNj3S4IP6Cpf+Fb+Nv+f1P/AAMaooVKtKHL%0AyM+6zXC4LH4j26xMY6JW9Pme1+dF/wA9E/76FHnRf89E/wC+hXin/Ct/G3/P%0A6n/gY1H/AArfxt/z+p/4GNWv1mr/AM+2ed/Y2C/6C4/18z2vzov+eif99Cjz%0Aov8Anon/AH0K8U/4Vv42/wCf1P8AwMaj/hW/jb/n9T/wMaj6zV/59sP7GwX/%0AAEFx/r5nsOpTRf2Xd/vE/wBS/wDEP7pryL4Rf8jbf/8AXu3/AKEtRH4a+NWB%0ABvIyDwQbtuf0rpvh74I1fwzrM91qHkeW8BQeXJuOcg+ntWbdSrVhJxasdapY%0ATBYCvThXjNzSsl5M9Jooor0T5EKKKKAIbv8A485v9w1zHhH/AI+bn/dH9a6i%0A6UtayqBklTgVxFpBq9izNbRSxlhg/KDn8687FycK0J2bSvsYVHaSZ3lFcb9r%0A8Resv/ftf8KPtfiL1l/79r/hVfX4/wAkvuH7ZdmdlRXG/a/EXrL/AN+1/wAK%0APtfiL1l/79r/AIUfX4/yS+4PbLszsqK437X4i9Zf+/a/4Ufa/EXrL/37X/Cj%0A6/H+SX3B7ZdmdlRXG/a/EXrL/wB+1/wo+1+IvWX/AL9r/hR9fj/JL7g9suzO%0AyorjftfiL1l/79r/AIUfa/EXrL/37X/Cj6/H+SX3B7ZdmdlRXG/a/EXrL/37%0AX/Cj7X4i9Zf+/a/4UfX4/wAkvuD2y7M7CT/Vv9DXI+Gv+Q7P/wBc2/8AQhTD%0AdeIiCCZcH/pmv+FWPDdndQapJJPC6AxnkjvkVjKt7atTtFqz6oly55KyOroo%0Aor1ToCiiigBkv+pf/dNcn4W/5Cc3+7XWycxOP9k1wVr/AGjYztJbxOrHg/Lm%0AvOxsuSrTnbRXMKrtKLO/orjf7W1z+63/AH7o/tbXP7rf9+6r+0Kf8r+4ftl2%0AOyorjf7W1z+63/fuj+1tc/ut/wB+6P7Qp/yv7g9sux2VFcb/AGtrn91v+/dH%0A9ra5/db/AL90f2hT/lf3B7ZdjsqK43+1tc/ut/37o/tbXP7rf9+6P7Qp/wAr%0A+4PbLsdlRXG/2trn91v+/dH9ra5/db/v3R/aFP8Alf3B7ZdjsqK43+1tc/ut%0A/wB+6P7W1z+63/fuj+0Kf8r+4PbLsdg/+rb6GuQ8N/8AIem/3H/mKadV1wgj%0Aa3P/AEzFS+G7edNWeSWNlBjbkjHORWE66rVqfKnoyHPmkrHXUUUV6x0hRRWN%0A4m15dA0xZlj825mkENvF/fc9KANmiuQs7Dxfb+LreefVVudIkt83ELRovlye%0AiYGcfUmuvo6AFFFFABRRRQAUUUUAFFZfiPWovD2gXmqSruECZC/3m6AficVi%0AXmvaloUWlXuoyxTW99OkEqKm3yi/3Sp9M4HOetC1dv61B7X/AK0OvoqOaQpb%0ASSINzKhYAd+K8+m1L4j2VmJpE0Jt77Y0eOTe2TwOGAzR1sB6LRXGaxqfi2C7%0A02205dKNw9u0l0tyGCZBUfL82e/rXS6QNQ+wI2pzW8tyxyxt1IQewySf1oAv%0AUUUUAFFFFABRRWN4m15dA0xZlj825mkENvF/fc9KANmiuRs9P8XW/i22nn1Z%0AbnSJICbiFo0URyeiYGcfUmuuo6AFFFFABRRRQAUUUUAFFUNa1SHRNFvNSnGY%0A7aJpCPXA6VzVz4i1TSdHsNcvniktbmWNJoFTHlLIQqlT14JGc5oWrt6fiH9f%0AcdpRUUspW1eWMbiELKB34zXn82pfEeysxNImhNvfbGjxyb2yeBwwGaOtg6XP%0ARaK4zWNT8WwXem22nLpRuJLdpLpbkMEyCo+X5s9/Wul0gah9gRtTmt5bljlj%0AbqQg9hkk/rR5gXqKKKACiiigAoorG8Ta8ugaYsyx+bczSCG3i/vuelAGzRXI%0AWdh4ug8W209xqq3OkyW5NxA0aKI5PRMDOPqTXX0dACiiigAooooAKKKKACiq%0AOs6pFoui3mpTjMdtE0hHrgdK5m68RappGjWGuXzxSW1zLGk0CpjylkIVSp68%0AEjOc0LV2/rUP6+47SimeYDF5i8gruHvXn0+pfES0tJLoponklyIkmR/MOThR%0Aww5o62A9EoritT1TxfDLpVvZJpP2uW3eW7W4DCMEFQNvzZ/i9a6fSBqH2BG1%0AOa3luWOWNupCD2GST+tAF6iiigArzfxJrWmXPxM0iyvL+CG30yMzv5jYBlbh%0AR9cDP416RWXP4b0S6R1udIsZxISzGWBXLE9ySKOqfYOjRI+uack1rCt0kkl0%0AcRLGd27347Vg6k0t78QrGwiuJ0hhtTPcpHKyhskhQcH2NbOk+G9G0IN/ZemW%0AtqW6tFGAfzqKx8PCz8Q3usNfzzzXSqjI6oFVV6AYANHVMOjG3vie1tXvNkUk%0A0dj/AMfUidI+M49zjtWVLcNqvj6yhtrmYWkVn9omVJGCuT93IB+hrQl8JW0v%0A22L7VOtpeyma4t1xhyeozjOD9ank8PhNVbUbC5a0lkiWGQKilSo6YBBxihdG%0A/wCtP8wfVL+tf8gtoDP4lvLgTTeXCqxiPzDs3EZJx0rWnANvIGV2G05CEgn6%0AY71FZWUdjAY0ZnZmLO7dXY9Sas0W0sHW5wRsrDJ/4k3ib/wNuf8A45V/RrW0%0Aj1SJotM12FxnD3V1O8Y47hnI/SuuooQM5D4n2c178P8AU44FLOgSUgdcK4Y/%0AoDWZ46I1Hw14ftrUiSWe/tSgU5OFYMT+ABr0FlDKVYAqRgg9DVG20TTbS4E8%0AFnEkighCF+4D12/3fwoWj+af3A9vk194+DUbOS9fTo7mNruFA0kIPzKOxIrn%0AF1A6j8Tn04nMOnWXmhe3mOQAfwGR+NdSlnbR3T3SW0K3Eg2vKEAdh6E9SKjj%0A06ziv5b+O2jW6lUJJKF+ZgOgJoW6bDo0jInNjf8Ai1rS70wu1va71uZWzGQx%0AGV2ngnjOazPAU9y954ht9zNp0F9ts2JyACvzAewOPzrqrzTbS/2/aYFcrwG6%0AEDuMjnFS21rBZwLBbQxwxL0SNQoH4Chaf153B6/15EtFFFABRRRQAV5x4k1n%0ATbj4maRZXl9DDb6bGbh/MbAMrcKPrgZ/GvR6y7jw3ol2ki3Ok2U/mMWcywK5%0AJPckijqn2Do0SPrmnLNawrcpJJctiJYzuz78dqwNTeW9+IVhp8VxOkMVsZ7l%0AY5WUNkkKCAfUVtaT4a0XQt39l6Za2rN1aKMA/nUVj4eFn4ivNZa/nnmukVCj%0AqgVVHQDABo6oOjG3vie2tWvfLiknjsRm6kTpHxnHucdqypbhtV8fWUNtczC0%0Ais/tEypIwVyfu5AP0NaE3hG2l+2xfap0tL2Uy3FuuMOT1GcbsH61PJ4fCaq2%0Ao2Fy1pLJEsMgVFKlR0wCDjFC6N/1p/mD6pf1qgtoDN4lvLgTTeXCqxiPzDs3%0AEZJx0rWnANvIGV2G05CEgn6Y71FZWSWMJRWZ2Zizu3V2PUmrNHSwdbnBGysM%0An/iTeJv/AANuf/jlXtGtbOPVImj0zXoXGcPc3U7xjjuGcj9K6+ihAzmfiFZT%0Aah4C1i3t1LSm3ZgB1OOf6Vzni511D4ZaZbW5DzXMtqkar1JEik/kAa9JIyMH%0ApWfDoemW9ytxFZQrIpJTC8IT1KjoD9KSWv3fgNv9fxHWl/aNcf2atzG15BEr%0ASQg/Mo6ZIrnV1I6j8T200nMOnWRm29vMdgAfwG4fjXVJaW0d090lvEtw67Xl%0ACAOw9CepFRx6bZxahLfx20a3cqhJJgvzMo6Amn1TfmLo0Y9w1jf+LjaXell2%0At7bctzK2YyGPK7TwTx1rN8BT3L3niG33M2nQX22zYnIAK/MB7A4/OuqvNNs7%0A/b9pgVyvAboQPTPXFS21rBZwLBbQxwxL0SNQoH4Chaf153B6/wBeRLRRRQAU%0AUUUAFeceJNZ024+JmkWV5fQw2+mxm4fzGwDK3Cj64Gfxr0esu48N6JdpItzp%0ANlP5jFnMsCuST3JIo6p9g6NEj65pyzWsS3SSSXLYiWM7s+/HasLU2lvfiBp9%0AhFcTpDFbNPcrHKyhuSFBwfUVsaT4a0XQt39l6Za2rN1aKMA/nUVl4e+yeIbv%0AWWv555blBGY3VAqqOgGADR1QdGNvfFFrave+XFJPHYjN1InSPjOPc47VlTXD%0Aat4+sobW5mFnFZ/aZlSRgr5+7kA/Q1fl8I20v26L7VOtpfSmW4t1xhyeozjO%0AD9asSeHwmrNqNhctaSyRLDIFRWUqOnBBxihdG/60/wAwfVL+tV+gW0Bm8S3l%0AwJpvLhVYxH5h2biMk46VrTgG3kDK7DachCQT9Md6isrJLGEorM7MxZ3bq7Hq%0ATVmjpYOtzgjZWGT/AMSbxN/4G3P/AMcq9o1rZx6pE0ema9C4zh7m6neMcdwz%0AkfpXX0UIGc18QLOa/wDAWs28ClpWtmIA6nHP9K5vxe66h8MNNtrch5rmW1SN%0AV6kiRSfyANekkZGD0rPh0PTLe5W4isoVkUkpheEJ6lR0B+lJLX7vwHf9fxFt%0Ab+0W4j0v7TGb2OBXaHPzBeBnHpmufvNRN78TLHRif3NpZveMvq5Kqp/AFq6k%0AWlsLs3Yt4hcldhm2DeV9N3XHtUY06zGpHURbRi8aPyjNt+YrnOM+lVfVNk9G%0AkY9w1lfeLhZ3emF2t7UutzK2Y8M3K7TwT8oOazPAU9y994ht8s2nQXu20YnI%0AA2jcAfQH+ddXeabaX4X7TArlejdCB3GeuKltrWCzgWC2hjhiXokahQPwFJaf%0A153G9f68iWiiigChrWrW+h6Rcahcn93CucdyewrljH41ur/RdSt75EtJ233l%0Ai0SbY4zyMNjcTjrz1qp8SNSspNQ0PQ7q8igimuRcXDSNgCNOmfqf5V2MuvaX%0ABZx3P2yJ4pGCRmNg24noABRH+bz/AK/ryCXbyNKiuVTUrrX/ABLf6bbzPbWG%0AnFUndOHlkKhtoPUAAjpU1go/4SMCz1dprOOIiS2eYyMJD3JOT6ULWwPQ6Siu%0ASsZZr7x/qirdTiysY41MYlbYZSAx4z6EcVffxXZKDMEkazWf7O1yPu787ePU%0AZ4zQtbeYPS5vUVyelyz3/jrVyLmf7FZBIxF5rbPMIBJxn9K09Fid7q+uzcTy%0AI87JGjyFlUA44FC1B6GzRVDWVR9MlDpduvHy2jssh+hUg1yH2a1/58PFv/gb%0Ac/8AxdAHfVieKNf/ALA06GREV7m5njtoFboXdgMn2HJ/Cq/hyKGO4lMdvrUR%0A29dQnlkU89t5PNZPxLgka20K6UExWurQSSn0UttyfxIo6pd2vzBbP0f5Gk2t%0A3mkeIdN0vUpY501EOIpFXaVkUbtv0wD+VbWrzXUGk3Mlkge6CERKRkbjwM/j%0AXIeLImvvHXg+O3+cxTyzuV/hQRsMn8WH512drqNnfSTx2tzHM8D7JVRslG9D%0ARvH7w2f3HDHVviJaSWcNxaaG8k7hcKX3Y7nhuwrR1LVPFo8QS2mkw6W9tHDG%0AZGuWYFWbdkjBGRxUnh3Uf7Z8X6/M3K2DJaQ+3Ut+fy/lUix6Zqup6093ppje%0A3Cwmec5VwATuQHpjPWk3pf5hbW3yOisxcLaRi7kSS4x87RjCk+1T1yXw4vL6%0A98JJJfs7lbiVIXfq0Yb5T79x+FdbVNaiQUUUUhlDWtWt9D0i41C5P7uFc47k%0A9hXLbPGt3f6Lqdvexx2c7b7yxaNNscZ5GGI3FsdeetVPiRqVlJqGh6HdXkUE%0AU1yLi4aRsARp0z9T/Kuwl1/SoLOO5F5E8MjBIzGwbcT0AAoj/N5/1/XkEu3k%0AadFcqup3OveJb7TLeZ7aw0/as7pw8jkBtoPYAEdKlsFH/CSKtlrDTWccR8y2%0AeYyN5nqScnHTiha2B6HS0VyVjLNfeP8AVFW6uPsVjHGpiErbDKQGPGfQjir7%0A+K7JQZgkjWaz/Z2uR93fnbx6jPGaFrbzB6XN6iuT0uWe/wDHWrkXM/2KyVIx%0AF5p2GQgEnGf0rT0WJ3u7+7NxO8bzMkcbyFlUA44FCBmzRVPVFVtNnDpcupXl%0AbZmWQ/7pXkH6Vxv2a1/58PFv/gbc/wDxdAHfVieKNf8A7A06GREV7m5njtoF%0AboXdgMn2HJ/Cqnh2GGO9cx22uRHZ1v7iZ0/AOxGazPiXBI1toV0oJitdWgkl%0APopbbk/iRR1S7tfmC6+j/I0m1u80jxDpul6lKk6akHEUirtKSKN23jtgH8q2%0AtXmuoNJuZLJA90EIiUjI3HgZ/GuQ8WRNfeOvB8dv85inlncr/CgjYZP4sB+N%0Adla6jZ30k8drcxzPbtslVGyUb0NFrx+8Nn9xw51b4iWklpDc2uhvJO4XCl92%0AO54bsK0NT1TxcPEEtppMOlvbRwxmRrlmBVm3ZIwRkcVL4d1D+2fF+vzNytg6%0AWsP+zwS35/L+VSJHpmq6prT3emFHtwsJmnOVcAE5UHpjPWk3pf5hbW3yOish%0AcrZxC7kR7jHztGMKT7e1T1yfw5u7688KB75nbZcSxwu/Vog3ynPf0/Cusqnu%0AJBRRRSGVNT1CDStNuL65bbDAhdjXHO3jTVBpGr6feJb288geawaNNqwnkEsR%0Au3Yx3qL4o6lbfZtM0Oa7jtxfXKmZnbAWJeua6w69pFtpiXSXsLWwwiGNgc9g%0ABiiP83n/AF/kD7eRq0Vyq6nc694lvdNt5ntrDTwgndOHkdlDbQeoABHSprFR%0A/wAJGq2WrtNZxxHzLZ5jI2/1JOTihagzpKK5Gxlmv/iDqaLdXAsrGKMGMSts%0AMpAPTPoelX5PFlkgM4SRrJZxbtcj7u/dt49RnjNC1t5g9Lm/RXJ6XLPf+OtX%0AIuZzZWSpGsXmtsMhAJOM/pWno0Tvd392bid42mZI43kJVQDjgUIGbNFU9UVW%0A02cOly6leVtmZZD/ALpXkH6Vxv2a1/58PFv/AIG3P/xdAHfVjeKNeTw7orXp%0AQPI0iQxIejO7BR+GTVLw9DDHeuY7bXIjs639xM6fgHYjNVPiNpV5qui2KWUT%0ASvDqFvKyr/dEgyfw60Pp8vzGuv8AXQnm1y+0XVdJttTkjmh1NzErKu0xSbSw%0AHuMAj8q3dXmuoNJuZLJA90EIiUjI3HgZ/Gud8XaJe6xe+G2tVQrZ36zzEtjC%0ABGHHryRXTWuo2d7LPFa3Mcr27bJVRslG9DRun8/u0EtGvl9+pwx1X4iWklpF%0Ac2uhu87hcKX3Y7nAbsK0NT1TxaPEMlnpMOlvbRwxmRrlmBVm3ZIwRkcVJ4d1%0AH+2vGOvytymnslpD7ZyWP4/L+VTx2+n6vqusvdaU6vABCZ5vmWQAE5QdsZ60%0Ar6X+YW1t8joLIXK2cQu5Ee4x87RjCk+3tRXNfDu6v7rwqDf+YzJcSxwtJ1aI%0AN8p/p+FFU1qJM27rQNIvpJZLzTbW5eXG8zRK5OBjuKqaX4O8O6LdNc6do9nb%0AzE53xxKCPpxxRRSWmw3qMn0b7DqlzqdhdPbvdlfPjCBldgMBuehwAPwqbSdA%0Aj0y5v7wXDy3V+4kkldQCMKFAAHbiiihAxuieHRo8moO17LdPfTGaQyIq4J44%0AwOmKrW3hC2hhgs2uZZLG2m86O2ZRt3Z3DJ6nB9aKKAJBojWWp3clleyW638n%0AmSqEVsN3IJratLWKztY7eEYRBgZOTRRQtg6k1FFFABUc9vDdQPBPEksTjDI4%0AyCPcUUUAVbLSLDT5WltrdElYbS+Mtt9M+ntU9vZWto8r29tFC0zbpDGgUufU%0A46miigBlpptnYS3ElrbxxPcv5kzIuC7epqK80bT9Qk33NskjEYbI+8PQ+o9j%0ARRQBciijgiWKJFSNRhVUYAHsKfRRQAUUUUAZ11oGkX0ksl5ptrcvLjeZolcn%0AAx3FVNL8HeHdGumudO0ezt5ic744VBH044oooWmwPUbPo32LVLjU7C6e3kus%0AefGEDK5AwG56HAH5VLpPh+PTLq/vRcPLd3zh5JXUAjChQAB24oooQMbofh7+%0AxpNQka9lunvpjNIZEVcE8cYHTFVrbwhbQwwWbXMsljbTedHbMo27s7hk9Tg+%0AtFFHYCQaI1nqd3JZXslut+/mSqEVsNjkgmtq0tYrK1jt4RhEGBk5P40UULYO%0ApNRRRQAVHPbw3UDwTxJLE4wyOMgj3FFFAFWy0iw0+Vpba3RJWG0vjLbfTPp7%0AVPb2VraPK9vbQwvM26Ro0Clz6nHU0UUAMtNNs7CW4ktbeOJ7l/MmZFwXb1NR%0AXmjafqEm+5tkdiMMSPvD0PqPY0UUAXIoo4IliiRUjQYVVGAB7Cn0UUAFFFFA%0AGfdaHpV9M8t5p9tcu4CsZog/A7cjpVLTvBnhvSb03lhotlbzk5DxwqNp9uOK%0AKKFpsG4lxo32PVbjVLG6e3kutonjCBlcgYDc9DgD8ql0nQI9Mur+9Fw813fM%0AHklZQCMKAAAO3FFFCBjdD8OjRptQla+lunvpjLIZEVcE+mB0xVW38IW0MMFk%0A1zLJYW83nR2xUbc53DJ6nB9aKKOwEo0R7LU7qSyvpLdb9/MlUIrYbHUE1tWl%0ArFZWyW8IwiDAyck/WiihbB1JqKKKACogd88iEDCgY/GiigCneY0+0i8oFj5i%0AqC7EnBPOTnJqzBaW1q80sMEUck7b5WRAC59Se9FFADLPTLKwluZbW2jie5fz%0AJmRcF29TXNToZL7U0a4vSZLgLxdyKEXA4UKRt/CiigC34WnaXS5TumIW4dR5%0AszynA46sSaKKKAP/2Q==)

在上图的情况下，GIoU损耗退化为IoU损耗，而提出DIoU损耗仍然可以区分。 绿色和红色分别表示目标框和预测框

## 3.3.总结
- IOU：从IOU误差的曲线可以发现，anchor越靠近边缘，误差越大，那些与目标框没有重叠的anchor基本无法回归。
- GIOU：从GIoU误差的曲线我们可以发现，对于一些没有重叠的anchor，GIOU的表现要比IOU更好。但是由于GIOU仍然严重的依赖IOU，因此在两个垂直方向，误差很大，基本很难收敛，这就是GIOU不稳定的原因。

## 3.4.DIOU
$$
\begin{cases}
DIOU&=IOU-\frac{\rho(b,b^{gt})}{c^2} \\
\mathcal{L}_{DIOU}&=1-IOU+\frac{\rho(b,b^{gt})}{c^2}
\end{cases}
$$

$b，b^{gt}$分别代表了anchor框和目标框的中心点，且$\rho$代表的是计算两个中心点间的欧式距离。$c$代表的是能够同时覆盖anchor和目标框的最小矩形的对角线距离。因此DIoU中对anchor框和目标框之间的归一化距离进行了建模。直观的展示如下图所示。

![Images](data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsL%0ADBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/%0A2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy%0AMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCADrATMDASIAAhEBAxEB/8QA%0AHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUF%0ABAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkK%0AFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1%0Adnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXG%0Ax8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEB%0AAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAEC%0AAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRom%0AJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOE%0AhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU%0A1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3+iiigAoq%0AI3UAODKn50n2qD/nqn50roCaioftdv8A89k/Oj7Xb/8APZPzpc0e4romoqH7%0AXb/89k/Oj7Xb/wDPZPzo5o9wuiaioftdv/z2T86PtUH/AD1T86fMh3JqKh+1%0AQf8APVPzo+1Qf89U/Oi6AmoqH7VB/wA9U/Oj7VB/z1T86LoCaioftdv/AM9k%0A/Oj7Xb/89k/OlzR7iuiaioftdv8A89k/Oj7Xb/8APZPzo5o9wuiaioftdv8A%0A89k/Oj7VB/z1T86d0O5NRUP2qD/nqn50faoP+eqfnRdATUVD9qg/56p+dH2q%0AD/nqn50XQE1FQ/a7f/nsn50C7tycCZM/WlzR7iuiaiiiqGFFIzqv3iB9ab50%0Af99fzoAfRTPOj/vr+dHnR/31/OgB9FM86P8Avr+dHnR/31/OgB9FICGGQcil%0AoAKKKKACiiigApG+4fpS0j8I30oA4nU9STTLJpTy5Zgi+pzVDSPEsV3thusR%0AzdA3Zqtr9m1uO5gkhJjikZQ57n2rl9V8PXOnlpY1MtuOd4H3frXp4KhQnT9j%0AV0meBjsVi6UliKHvU+353/zOj1f/AI+U/wBys+s3S9XjWVItS8ySADaGB5UV%0A31poOlX1us9tO0kbDhlavm8xybE060pW0ezN8LiqeNvKm9e3U5Oiuz/4RWx/%0AvSfnR/witj/ek/OvP/s2v5HX7CZzek/8fZ/3a26uw+HLW3ffG7g9Oasf2TH/%0AAM9W/IV62DpSo0uSW500ouMbMyqK1f7Jj/56t+Qo/smP/nq35Cuq5oZVFav9%0Akx/89W/IUf2TH/z1b8hRcDjtV/4/f+AiqNdvN4btbiTfI7lsYqP/AIRWx/vS%0AfnXi18BWnUlJbM5J0ZOTaOMors/+EVsf70n50f8ACK2P96T86x/s2v5E+wmc%0A7pH/AB8P/u1s1eg8O2tuxaN3BIxzUsumRRRPI0r4RSx4HavXwlKVKkoS3Omn%0AFxjZmZRXntx8afCNvcywOL7dG5Q4j7g4qP8A4Xf4P9L/AP79103ND0aivOf+%0AF3+D/S//AO/dH/C7/B/pf/8Afui4HUan/wAfz/QfyqC2/wCPmL/eFcVqfxh8%0AKzuslsl4WP3tyVTh+L2gJMjmO5wCD9yvAr4Ws6zko6XOOdOXO3Y+jKK8k/4a%0AF8J/88bz/v3R/wANC+E/+eN5/wB+6+gOw9J1b7sf1rLrg7j4+eELkAPDe8ei%0AVB/wvHwZ/wA8r7/vigD0OivPP+F4+DP+eV9/3xR/wvHwZ/zyvv8AvigD0Oiv%0APP8AhePgz/nlff8AfFH/AAvHwZ/zyvv++KAPYbH/AI84/pVivIYv2gfCMMYR%0AYb3A/wBin/8ADQvhP/njef8AfugD1uisnw34gtPE+hW+rWQcW84JUOMGtagA%0AooooAKiuiRaTEddjfyqWmyANGwPQgihOwmrqxw/h4D+xYG/ibcW+u410WmKr%0AySKwBUrgg1y+jyG2vLvTpONsheL3UntXVaV/rn+lb4j+I5LrqvnqcuBaeHjH%0AsrP1WjOa8ReBlk33WlKFbq0HY/SuU0vWNQ8PXjCPcuDiSF+AfqK9mrD17wxZ%0A63GXIEdyB8soH867sNmGnsq+sTy8bk/ve3wj5ZLp/W35D9D8SWWuRYjbZOB8%0A0THkfT1rZrxe/wBM1Hw9fL5gaN1OY5U6H6Guy8O+OI7jZa6oRHL0E3RW+vpS%0AxOX2XtKGsR4HOLy9hilyzX9fI7aikBDAEHIPQilryz3gooooAKKKKACiiigA%0AooooAKgvf+PG4/65N/Kp6gvf+PG4/wCuTfyoA+EtV/5DF7/18Sf+hGqlW9V/%0A5DF7/wBfEn/oRr3f9njS7C+0jVpLq0hmcTqAzqCQMCgD59q7Z6ZPfECFo8nn%0ABcZ/LrX3B/wj2j/9A21/79ivPfihaWFvFpumWNnDHPdS5PloAcdB/Wsq83CD%0Akj0Mrw0MTio0qi0e+ttNz5bu7ZrO5aB2VmXGSvSoK+wdd8N6ZYaZp0AsoC0a%0ABGbYMsQBzWLb6RpxuIwbKD7w/gFcdTH+yn7OUbvQ83EVqca0lTXu309Oh8sU%0AV92/8I9o/wD0DbX/AL9ij/hHtH/6Btr/AN+xXolHwlRX3b/wj2j/APQNtf8A%0Av2KP+Ee0f/oG2v8A37FAHwlRX3b/AMI9o/8A0DbX/v2KP+Ee0f8A6Btr/wB+%0AxQB8JUV92/8ACPaP/wBA21/79ij/AIR7R/8AoG2v/fsUAfCVFfZ3jXwjpuo+%0AD9Rs7aytIp5kVEkMfCksMHivCv8AhQmu/wDQV0//AMf/AMKAPafg1/yS/Sf9%0A1v513tcp8OtGm8PeC7PSriRJJbYlGePO09+M11dABRRRQAUjfcP0paRvuH6U%0AAcBqiRyFZ4JVS7gZtue4z0NbfhfVItRL4+WZRh4z1B/wrmbn/j6m/wB9v51o%0AaPow1BJp4Jmt7uMgxyr/ACNceCzBVpLD1Vbs+3r5HmXqRqc9JXvuu/mvP8zu%0A6K56y16a2uRYa1GLe46JN/yzl9we1dCDkZFehUpyg7M7qNaFVXj811XqVr2w%0AttRtmguohJG3Y9q818Q+DbnSi1xaBp7XrxyyfWvU6QgMCCAQexrbDYuph37u%0A3Y5cdl1HGR99WfRnlXh7xhc6QVguN01p0x3X6V6ZYaja6nbLcWsqyIfTqPqO%0A1ct4i8ER3e+500LHP1MXRW+npXE2d/qXh2/OzfDIpw8bjg/UV6U6FHGx56Ok%0Au39fmeJSxWKyuapYlc0Oj/r8j2misDw/4ptNaQRkiG6A5jY9fp61v141SnOn%0ALlmrM+lo16deCnTd0FFFFQahRRRQAUUUUAFQXv8Ax43H/XJv5VPUF7/x43H/%0AAFyb+VAHwlqv/IYvf+viT/0I17X8D/ENp4c8JaxdXJLO1yFiiX70jbRwK8V1%0AX/kMXv8A18Sf+hGvZfgx4bW71WDzl3Q26/aHB6bj0/l+lY1qjiko7s9HLsJG%0Au51KvwQV359l8z2axsdd1a1F5qOoSWLSDdFbW/Hlg9Nx7muH003viP4pQxag%0Ayy/2VuQuo4YKTgkepzXp+uatBomj3F9O2FjX5R3J7AVwvwmtZLkarrtwP3t3%0AOQD+px+JP5VjUinUjC/mz08FXlHCV8U4pK3LGytZve3yOl8X/wCrtvqa5q2/%0A4+Yv94V0vi//AFdt9TXNW3/HzF/vCvJx3+8v5Hx1X+Iel0UUV9EdoUUUUAFF%0AFFABRRRQBn63/wAgmX/eT/0Naya1tb/5BMv+8n/oa1k0Aa2j/wDHpJ/11P8A%0AStCs/R/+PST/AK6n+laFABRRRQAUjfcP0paRvuH6UAea3P8Ax9Tf77fzrpfC%0AP+ruPqK5q5/4+pv99v510vhH/V3H1FfO4L/eV8zipfxDdvrC21G3MNzEHQ+v%0AUfSsDdqPhhgG8y+0v+8OZIh7+orqKQgMCGAIPUGvp6dVxXK9V2Na2HU3zxdp%0Ad/8APuiCzvba/t1ntZlljbutWK5280Oeyna+0RxFL1e3b7kn+Bq3pWvQ6gxt%0A5ka2vV4eCTg59vUVU6Sa56eq/Fev+ZNPENS9nWVpfg/T/Lc16yNb8O2WtwkT%0ALsmA+WVRyP8AGteis4TlCXNF2ZtVpQqxcKiumeNatomoeH7sFwwUH93MnQ/4%0AV1HhzxyCUtdVOOyz/wDxX+NdvdWsF5A0NxEskbDBVhXnPiLwTNY7rnTg01v1%0AMf8AEv8AjXsU8TRxcfZ11aXc+arYHE5dN1sI7x6r+t/zPSkkSVA8bBlYZBBy%0ADTq8j0HxVeaLIInzNbZ+aNjyPpXp2mataatbCa1lDD+Je6/UVwYrBVMO7vVd%0Az18BmdHFqy0l2/yL1FFFcZ6QUUUUAFQXv/Hjcf8AXJv5VPUF7/x43H/XJv5U%0AAfCepnbrV4fS4c/+PGvbPhX4ttdEuJJJIZZ4rmJU/cLuZWHbH414lqv/ACGL%0A3/r4k/8AQjXu3wAntLLwvrl9dxqyW8wYkqCQNo6ZrCtT5mpJ2aPUy/GqjGpQ%0AnDmjUsrXs730Ow8czajf+Ep9UuraSCJmEdvbH7yqervjoeOnatX4evcHQdPt%0A7W2eCzhQmaWZMGVyScKPQZ612FtNDqWnw3HlZinjWQJIozgjIyKsKiooVFCq%0AOgAwBSjR9/2l+hdXMr4V4X2drSv5LS23fzOa8X/6u2+prmrb/j5i/wB4V0vi%0A/wD1dt9TXNW3/HzF/vCvFx3+8v5HzdX+Iel0UUV9EdoUUUUAFFFFABRRRQBn%0A63/yCZf95P8A0Naya1tb/wCQTL/vJ/6GtZNAGto//HpJ/wBdT/StCs/R/wDj%0A0k/66n+laFABRRRQAUjfcP0paRvuH6UAea3P/H1N/vt/Oul8I/6u4+oqJYY2%0A3ExqTuPb3rT0hFSVwqgAjtXmYbAunUVW5zwpOMua5sUUUV6Z0BWZquiW2qKG%0AbMVwv3J4+GU/1rToqoTlB3i9SKlONSPLNXRzVvrN3pE62euDKE4jvEHyt/ve%0Ahro0dZEDowZTyCDwaZcW0N3C0M8ayRsMFWFc49rqHhpzLY7rvTur27H54x/s%0Anv8ASt7QrbaS/B/5fkcl6mG+L3od+q9e6/E6ijrVPTtUtdUtxLbSZ/vKeGU+%0AhFXK55RcXZ7nZCcZx5ou6ZyfiHwXb6kHuLLbDdddv8L/AOFcDHNqXh3UDgvB%0AMh5B6N/iK9qry/4r67HZfZrFLWN55BuMrDlR6Cu+hmfsYONfWJ5tXh6WPrx+%0Ap+7UfyX/AAC1bfEPUNYkj0/SLCE3p4eaeTbEp9uMmtT7N4/A8wXujl/+eex9%0Av514paa2bQgJDhQcgq2CK9I8L/FG3TZaas7eX0WYjlfr615reGqrnhV17bW9%0AL7n1csLmeCl7KWDvBbSspt+crN2+SsvxOiHiPxPpvGq+HPOjH3p7KYN/46cG%0Ardr4/wBBnYJPLNZSnjZdQlD/AFFbtnqljfxq9pdwyq3I2uM/lS3Wm2V6pW5t%0AIZQeu9ATWajNfDK/qYyrYaTtWo8r/utr8Hf9B1tfWl4oa2uYpQf7jg0t7/x4%0A3H/XJv5Vztz4A0SRi9qtxYydd1rMV5+hzVG68P8AifTbWY2HiEXMIRv3N5Dn%0AAx/eBp881vH7ifq+Fqfw6tvKSa/FXX5HyFqv/IYvf+viT/0I16h8NNSS28FX%0A+ngnzb+/SMKB1UBd38xXA3/9kLqN00v2lpfNfeowF3ZOfwr1T4U6DB4k8Oar%0Ac2UPk3lpMptfmyMgZ5+v9KzqTlODUYs68FQoYfEQqV6i0eiWuvS/RJdT6Kgi%0AWCCOFPuxqFH0AxUlc/4c8UW2s24hnP2fUofknt5OCGHUj1FdBW8ZKSujy69G%0ApRqOFRWZzPi//V231Nc1bf8AHzF/vCul8X/6u2+prmrb/j5i/wB4V8/jv95f%0AyPMq/wAQ9Looor6I7QooooAKKKKACiiigDP1v/kEy/7yf+hrWTS/EC9m07wJ%0Aq15bMFmgiDoSM4IYV8w/8Lg8Yf8AP7F/36oA+s9H/wCPST/rqf6VoVx/wv1S%0A61nwFYajeuHuJ8s7AYyc12FABRRRQAUjfcP0paRvuH6UAcwnQ/7x/nWlpX+u%0Af6Vmp0P+8f51paV/rn+lTD4UJbGtRRRVDCiiigAooooAwtS0BjOb7Spfsl6O%0ATj7knswpdM18TT/YdRi+y3w42t91/dTW5VHUtJtNVg8u5j5HKyLwyn1BrojV%0AUly1fv6r/M454eUJOdDR9V0f+T8/vL1c34r8G2PiuCMTu0M8X3JU7exHeo47%0A+/8ADriDU91zY5wl0o+ZP94V0kE8VzCssLh0YZDA1nVo8q11i/uOjCY1qfNT%0AbjOP3o84f4N6WbMKmoXIue8hAK/98/8A165XWPhPrWnRPNaTRXka8kKCr/lz%0A/OvdaK4pYOjLofQUOI8xo/b5vVX/AOD+J8s7NR02U7TPbuDzsYqf0rc074h+%0AJdNwq35mQfwTruH9D+tey+IfB9prCNNCFhuv7wHyv9RXlWqeH/sV00F7aBHH%0AfGAfpiuilkyqq+Gq2fZ/5r/ImtxzCm+XNMIpR/mjZ/g9vvOg0z4zHAXU9MBP%0AeSB8f+On/Gupg+IvhvVLKZEvDDI0bAJMuCTj2zXnVh4Q0XWcQx3stheHoJAH%0Ajb6dCPzqLVPhZr9gjyw+TdRKCcxthsfT/wCvXHVhjsLLlqRuenh/9W81hz4a%0ApyN/L89PxPCNWVhq14xUgGdyCR1+Y19Afs2/8gPV/wDr4H/oIrzZxaXDNG4i%0AdlJVgcZBFdH4R8TXfgvzl0uKDyp2DSRyAkE/nUxx8dpqw63CdVrmw1RSX3f5%0Ao991nwppetOJpo3hul+7cW7bJB+Pf8axz4e8Vaac6X4hW5jH3Yb2HP8A48CD%0AWDpvxltJMLqOnSRHu8Lbh+R/xrr9N8c+HdUwINQRXP8ABJ8prVToVHeL1+48%0A+phs0wceSpTbiu6Ul+tvwOU8Q6v4jgSBNX0eNtpP720kyp/A81k2XiXTmuY/%0ANd7c7hkSpiu68Vyxyw2zRyK4yeVOa5dbW3uZ41ngjkBYA7lBrxsYrYhrfY8a%0AriMJOparRs+8W1+DuvyPSba/tLxQ1vcxSg9Nrg1Yrlrn4f6HKxe1S4sJTzvt%0AJmQ/lyKoSp4k8Igzi5bWdLXl0kXE0a+oI617vtJR+JfcdiwtCtpQqa9pK1/R%0A6r77HcUVT0vU7XV9PivbOTfDIMg9x7H3q5WqaaujgnGUJOMlZoKKKKZIUUUU%0AAcl8Tv8Akmuu/wDXv/7MK+La+0vid/yTXXf+vf8A9mFfFtAH2F8Gv+SX6T/u%0At/Ou9rgvg1/yS/Sf91v513tABRRRQAUjfcP0paRvuH6UAcwnQ/7x/nWlpX+u%0Af6Vmp0P+8f51paV/rn+lTD4UJbGtRRRVDCiiigAooooAKKKKAGyRpKjJIoZG%0AGCrDINc3NpN7ok7XeinfAeZLJzwf930NdNRWlOrKHo+hjWoRq2b0a2a3RnaV%0ArVtqsZ8slJl4eJ+GU1o1j6roEV9ILq2c2t8nKzR8Z9mHcVFpWszG6Omaogiv%0AVGVYfdlHqK0lTjJc9P5rqv8ANGMK06clTr9dn0f+T/pG7VLU9Js9WtjDdxBh%0A/C38S/Q1dorCMnF3i9TqnCM4uMldM8j17wreaJIZUzLbZ+WReq/WrejeM7i1%0AtJLK+zNCyFUf+Jcj9RXp7okqMkihkYYIIyDXB+JPBCqsl7pnygAs8J6fUV7N%0AHGU8RH2WJXzPm8RltfBz9vgnp1X9br8T5O1dJrbW7zcHjYzOR2OCxxXtHwQ8%0AO6d4q0HU/wC1o3meKYLHJvIZRtH+eap614fsdetysygSjhJV+8prsvgXpMmg%0A2+r2VxIhZpg8ZB+8uMZ/SuHGZc6WrXNHuevlOfOo7Qk4T7Xt9xc1L4MwNltN%0A1KRPRZ1DfqMVyOpfDPxJp+WW1W5Qd4Wyfyr6Dorx54KlLbQ+xw/FGPpaSakv%0ANfqrHy88usaU4ike6gKdEYnA/DpWjY+NNStJEaURzhTn5lwf0r2bxpBDPFbC%0AWJJBk8MoNcFL4T0q9mVfJMRY43RnH6dK8evBU6rhudcuI8sxUuTHYfXurP8A%0AyZv6d8YdKnwt9Zz2zdyp3j+lddp/ivQtXXFrqEDkjlGOD+teeaj8GZFy2m6l%0AuHZJl5P4jFcTrngzW/DyeZe237rOPMjO5fzr0vb4mn8cbnQsryXGv/ZqvK30%0A/wCA9fxPZfCVubHW9btoB/xL2m82DByoz1A9q66vEPhJNfjxI0SNIbQxsZAf%0Au5xxXt9deHkpU00fP51RlRxkoSabSWq66Lfz7hRRRW55QUUUUAcl8Tv+Sa67%0A/wBe/wD7MK+La+3/ABvpzav4N1LT0kEbXMaxhyMhcsOcV4F/woC//wCg9b/+%0AA5/+KoA9g+DX/JL9J/3W/nXe1y3w90V/D3g610mSZZntiUMirgN36V1NABRR%0ARQAUjfcP0paRvuH6UAch9tt0ZlaQAhjn861tEmSeSRo23ADBrjrn/j6m/wB9%0Av510vhH/AFdx9RXk4bGzqVVTa0OenVblynTUUUV6x0BRRRQAUUUUAFFFFABR%0ARRQAVla7pK6nZ5T5LqE74JB1Vh/Q1q0VUJuElKJFSnGpBwlszM0LUjqemrJI%0AMTxkxyr6MK065vTv9D8Zajar9yeJZ8ehzj+tdJWleKjP3dnr95jhJylTtLdN%0Ap/IKgvf+PG4/65N/Kp6gvf8AjxuP+uTfyrE6T4ys/Fd7oeuXkZYzWpuHzEx6%0AfMenpXp+ha4l9BHqOnTMpU4ypwVb0NeIar/yGL3/AK+JP/QjXvP7Pmn22peG%0AtZt7qJZENwOvUfKORXoYTHyo+5PWJ4+YZRTxP7yn7s+/f1/zPSPDvjeO82Wu%0ApFY5+gl6K319DXZAgjIOQa8q8Q+ELnSC09vumtOuQOU+tP8ADvjG50srb3Za%0Aa16DJyyfT2ror4GFaPtcM/kcWFzWrhp/V8crPv8A5/5nW+L/APV231Nc1bf8%0AfMX+8K3PEN9bajZWlxayrJGxPIPSsO2/4+Yv94V8Nj01imn5HrTkpT5ou6PS%0A6jmgiuIjFPEksbdVdQQfwNSUV9Cd5WtNPs7BStpawwA9fLQLn8qs0UUAFFFF%0AABRRRQBn63/yCZf95P8A0Naya1tb/wCQTL/vJ/6GtZNAGto//HpJ/wBdT/St%0ACs/R/wDj0k/66n+laFABRRRQAUjfcP0paRvuH6UAea3P/H1N/vt/Oul8I/6u%0A4+ormrn/AI+pv99v510vhH/V3H1FfO4L/eV8zipfxDpqKKK+iO0KKKKACiii%0AgAooooAKKKKACgkAEk4AorI10XtxHFYWaMBcHEsw6Rp3/E9KuEeaVr2M6s/Z%0AwcrXOct5NS1rxLeXmlkQwhRB9odewPYetbI8MzyfNPreoO/+zKQB9BWzZWcN%0AhaR20ChUQYHv71YroqYp3tT0S08zjo4CKjes7t6vV2u+yOdOiavafNY61NIR%0A/Bc/ODUM3iC4s4ZbXWrQ27OjKk6cxscetdRVXUoY59NuUlRXXy2OGGe1Qqyl%0ApUV/wZo8K4a0ZNeT1X4/ofC2q/8AIYvf+u7/APoRr6B/Zt/5Aer/APXwP/QR%0AXkl9caZr+oXNreBLO+SVkjuBwsmCQA3v717P+z/ptzpWn6zbXSbX89WB7Mu0%0AYI9qmdKy5ou8f63LpYhSl7Oa5Zdv8u6PZSAwIIyD1FcV4i8Dx3G+60xRHL1a%0AHs309K7aiihXnRlzQYYrCUsVDkqq/wCh4aHudOuGjYNG6n5kYY/StzTL6O6u%0AIh919wyprpfHGnwXK28hQLKcjeBzXnk0E9lMCcqQcq4retTwWbScH7tZfj/n%0A+aPl5wxGV1LL3qf9fce6UVwnh3xyJNlrqrBW6LP2P1rulZXUMpBUjII71jXw%0A9ShLlmj6fC4ylioc9J/5oWiiisDqCiiigAooooAz9b/5BMv+8n/oa1k1ra3/%0AAMgmX/eT/wBDWsmgDW0f/j0k/wCup/pWhWfo/wDx6Sf9dT/StCgAooooAKRv%0AuH6UtI3Kn6UAea3P/H1N/vt/OrWnatPpocQhSH65FbJhCswdBu3HP51VvbGa%0A6RVt4tzA5OK8WWBqUk6sJa+hyui4rmTE/wCEqvf7kf5Uf8JVe/3I/wAqpf2H%0AqP8Az7N+Yo/sPUf+fZvzFc/tcZ5kc1XzLv8AwlV7/cj/ACo/4Sq9/uR/lVL+%0Aw9R/59m/MUf2HqP/AD7N+Yo9rjPMOar5l3/hKr3+5H+VH/CVXv8Acj/KqX9h%0A6j/z7N+Yo/sPUf8An2b8xR7XGeYc1XzLv/CVXv8Acj/Kj/hKr3+5H+VUv7D1%0AH/n2b8xR/Yeo/wDPs35ij2uM8w5qvmXf+Eqvf7kf5Ux/GFzH98wr9eKwdRju%0A7a7jsEiP2qXhV6kVdg8NX8S5a1ZpD1YkV2KniYUlVquWuyW78/JHN9ZqzqOn%0AT6bvt5eppJ4uupBlBEw9RzTv+Eqvf7kf5Vi2nh7WFu5pJLRlU8AAjmrv9h6j%0A/wA+zfmKzxSxNKpywk5Ky/FbfIujVrTheSaLv/CVXv8Acj/Kj/hKr3+5H+VU%0Av7D1H/n2b8xR/Yeo/wDPs35iuf2uM8zbmq+Zd/4Sq9/uR/lUV14pvWs5xsj5%0AjYdPaq/9h6j/AM+zfmKiutE1EWkxNs2Njdx6UKrjPMOap5nydqLFtTu2PUzO%0Af/HjXr/wg1LUP7MuXhunEtvLhCTkbcD5T7V4/qAK6ldA9RM4P5mvYvglYXV5%0ApepNBEXCzAHH0Fe5WrVqNPmpbmuJpKpTs1/n8j1iDxtes3lTxxpKOxHB+lWv%0A+Eqvf7kf5Vmz+G764TD2jZ7HIyKozWOpaSAbyBzb9BLjO361xy9viFzULxl1%0Aj/8AIv8ATf1PP9tXo6VbuPf/AD/zNLUdWn1MIJgoCdMVRSGO4kSORQysQCDV%0AyHSb24iWWGHzI2GQysCDU8Gi6gs8bG2YAMCeRXl8mI9optO5u4ue6umZniLw%0AXPp265sQ01t1Kjlk/wAaqeH/ABZd6MywyEzWmeUJ5X6V60QCMHpXI+IvBUOo%0Ab7mwCxXPUp0V/wDA199Qx0KsfZYlXXf+vzPMxeU1KE/rGBdn2/y/yOi03VLT%0AVbUT2kodT1HdT6EVcrxW3udS8O6idu+CZDhkYcH/ABFekeHvFlrrKiGQiG7/%0AALhPDfSsMVgJUlzw1ideX5vDEP2VX3Z/n/XY6KiiivOPZCiiigDP1v8A5BMv%0A+8n/AKGtZNHxCu5rDwDq93bPsmhhDo3oQwr5Z/4Wp4w/6Cz/AJUAfXmj/wDH%0ApJ/11P8AStCuN+Fmo3WrfD/T769kMlxMCzue5zXZUAFFFFABRRRQBz91/wAf%0AUn+9VrSv9c/0qrdf8fUn+9VrSv8AXP8ASgDWooooAKKKKACiiigAooooA4DQ%0Aj/aHj+8uJfvRbigPbt/Wu/rjtT8Najba4dX0aSPexy0THGSev4VqQL4gvVCX%0ARt7OP+Ixnc5+nYV34lRq8s4yVrJeh5GBdTD89KcHdybv0d/M21dXztYNg4OD%0AnBp1Q21tHaQLFEMKPzJ9amrhdr6HrRvbXcKKKKQwqC9/48bj/rk38qnqC9/4%0A8bj/AK5N/KgD4S1X/kMXv/XxJ/6Ea+gv2bf+QHq//XwP/QRXz7qv/IYvf+vi%0AT/0I19Bfs2/8gPV/+vgf+gigD3KmyRpKhSRQykYII606igGrnNT6TeaJK13o%0ApLwk5ks2PB/3fStPStatdWjPlEpOvEkLjDIfpWlWPqmgxXsgubdzbXq8rMnf%0A6+tdKqRq6VN+/wDmcTozoPmobfy/5dvTb0NiiuesdemtrgWGtxi3uOiTD/Vy%0A/Q9q6HORkVlUpypuzOijXhVV4/NdV6mXrGg2WtQbLhMSAfLIByteYa14fvtA%0AuMuC0WfkmTp/9Y17HUc9vFcwtFPGskbDBVhkGurC46dB2esexwZhlVLFrmWk%0A+/8AmcD4d8ctGUtdVYsnRZ+4+td/FLHNGskTq6MMhlOQa868R+CJLXfdaYpk%0Ah6tF/Ev09ayND8TXuhy7OZIM/NE3b6eldlXB0sTH2uGevb+tjzMPmNfAz9hj%0AVp0f9b/mev0Vn6TrNnrFsJrWQE4+ZD95fqK0K8eUXF8slZn0kJxqRUoO6ZyX%0AxO/5Jrrv/Xv/AOzCvi2vtL4nf8k113/r3/8AZhXxbUln2F8Gv+SX6T/ut/Ou%0A9rgvg1/yS/Sf91v513tABRRRQAUUUUAc/df8fUn+9VrSv9c/0qrdf8fUn+9V%0AnSyBM+TjigDXopNy/wB4fnRuX+8PzoAWik3L6j86WgAooooAKKKTcPUUALRS%0Abl/vD86Ny/3h+dAC0Um5f7w/Ojcv94fnQAtFJuX1H50tABUF7/x43H/XJv5V%0APUF7/wAeNx/1yb+VAHwlqv8AyGL3/r4k/wDQjX0F+zb/AMgPV/8Ar4H/AKCK%0A+fdV/wCQxe/9fEn/AKEa+gf2cnWDRNWEzCPdOCN5xkbR60Ae50VF9qt/+e8X%0A/fYo+1W//PeL/vsUAS0VF9qt/wDnvF/32KPtVv8A894v++xQBHe2FtqFu0Nz%0AEHQ+vUfSuf3aj4YcBvMvdL9RzJEP6iuj+1W4/wCXiL/vsUhu7RgQbiEg9QXF%0AbU6riuV6rsc9bDqb54u0l1/z7oSzvba/t1ntZVkjbuKsVy97p62U7X+iXkEU%0A3V7dpBsk/wADV7SfElnqKmOVlt7pOHikYdfY9xTlSTXPT1X4r1/zJp4hqXs6%0AytL8H6f5bm1XMeIfB1tqqtPbbYLrrnHyt9a6H7Za/wDPzD/32KX7Vb/894v+%0A+xUUq06UuaDsy8Rh6eIhyVFdHjJGpeHdR/jt50P4N/iK9C8O+MbbVAtvdlYL%0AroM/df6GtXVLHS9XtzDdPEf7rhxuX6V5jrmgTaNPuSVZoCfkkRuR9R2r2I1K%0AGOjy1NJ/1/Vj5udDFZTLnpe9T6r+vzO1+J3/ACTXXf8Ar3/9mFfFtfTlzr1x%0Aqvhi98P38p8q6jEa3GMtHyOo79K46L4CwTxiSLxIXQ9CLYf/ABVeZiMLUoSt%0AJadz3sFj6OLhzU3r1XVHq/wa/wCSX6T/ALrfzrva5nwDoo8O+ErbSROZxbEp%0A5hXbu79K6auY7QooooAKKKKAMa6s5vNkk2/LnNUwSOhIro5E8yNkJxkYqh/Z%0AKf8APQ/lQBmb2/vH86N7f3j+daf9kp/z0P5Uf2Sn/PQ/lQBnRO3mp8x6+tdF%0AWeulorBvMPBz0rQoAKKKKACsG8Zhdy4Y/e9a3qoy6assrOXI3HPSgDJ3t/eP%0A50b2/vH860/7JT/nofyo/slP+eh/KgDM3t/eP50b2/vH860/7JT/AJ6H8qP7%0AJT/nofyoAoW7N9oj+Y/e9a6CqEemJHIr+YTg56VfoAKgvf8AjxuP+uTfyqem%0ATRiWGSMnAdSufqKAPg7Vf+Qxe/8AXxJ/6Ea+r/BtvCnhDSmWGNS1qhYhQM8V%0AzNz+zpplzdzTnWrgGV2cjyxxk59a9F0zwtcaXpltYxX8bR28YjUtCckD/gVA%0ADdif3F/KjYn9xfyq7/Yt5/z+w/8Afg//ABVH9i3n/P7D/wB+D/8AFUAUtif3%0AF/KjYn9xfyq7/Yt5/wA/sP8A34P/AMVR/Yt5/wA/sP8A34P/AMVQBxnxCJh8%0ABaxJEdjrbsQy8EcetfLP9qah/wA/11/3+b/GvsjXPB82u6NdaZcagiRXCFGZ%0AIeQD6fNXnf8Awzdpf/Qbuf8Av2P8aAPnv+1NQ/5/rr/v83+NXtK8T6npdz5i%0A3EkyNw6SOTuH17V7t/wzdpf/AEG7n/v2P8aP+GbtL/6Ddz/37H+NVCcoO8Xq%0ARUpxqR5Zq6PIbmKTWIGvNC1C6EgGZLN5m3L67eeRX0toShtBsS4BbyVyT1zi%0AuLt/2drG1mWaDX7uORTkMsY/xrsj4d1jQbdfsNwt9CB80bLhh9Oa3tCttpL8%0AH/l+Ry3q4fe8ofiv81+PqaWxP7i/lSNDE6lWjUg9QRVO1voZyEm1BLWbvHPb%0AlT/6FWj9lG3d/a9nj/c/+yrKVKpB6pm8K9KorxkmchrHhgrunsBkdWi/wqLw%0Aul+LxgrMlun+sVhxn0HvXQXl46uLfT7qO9umOAkcJwPcndxWvbaHfCBTJc26%0ASMMuFhPX/vqu+WLrKhyVVe+19zyIZfhpYtVaErW3tt6X/NGjo/8Ax6Sf9dT/%0AAErQqrYWrWdv5byB2LFiwXA/KrVeYe6FFFFABRRRQAUUUUAFFFFABRRRQAUU%0AUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRR%0AQBWubC0vBi4topPdl5/OqH/CK6Hu3f2fHn/eb/GtiitI1akVaMmvmZTw9Kbv%0AOKfqkV7axtbNdttbxxD/AGVxViiiobbd2aRioqyVgooopDCiiigD/9k=)

## 3.5.DIOU的优点
- 与GIOU loss类似，DIOU loss在与目标框不重叠时，仍然可以为边界框提供移动方向。
- DIOU loss可以直接最小化两个目标框的距离，因此比GIOU loss收敛快得多。
- 对于包含两个框在水平方向和垂直方向上这种情况，DIOU损失可以使回归非常快，而GIOU损失几乎退化为IOU损失。

In [ ]:
import torch

def Diou(bboxes1, bboxes2):
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    dious = torch.zeros((rows, cols))
    if rows * cols == 0:
        return dious
        
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        dious = torch.zeros((cols, rows))
        exchange = True

    w1 = bboxes1[:, 2] - bboxes1[:, 0]
    h1 = bboxes1[:, 3] - bboxes1[:, 1]
    w2 = bboxes2[:, 2] - bboxes2[:, 0]
    h2 = bboxes2[:, 3] - bboxes2[:, 1]

    area1 = w1 * h1
    area2 = w2 * h2
    center_x1 = (bboxes1[:, 2] + bboxes1[:, 0]) / 2
    center_y1 = (bboxes1[:, 3] + bboxes1[:, 1]) / 2
    center_x2 = (bboxes2[:, 2] + bboxes2[:, 0]) / 2
    center_y2 = (bboxes2[:, 3] + bboxes2[:, 1]) / 2

    inter_max_xy = torch.min(bboxes1[:, 2:],bboxes2[:, 2:])
    inter_min_xy = torch.max(bboxes1[:, :2],bboxes2[:, :2])
    out_max_xy = torch.max(bboxes1[:, 2:],bboxes2[:, 2:])
    out_min_xy = torch.min(bboxes1[:, :2],bboxes2[:, :2])

    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[:, 0] * inter[:, 1]
    inter_diag = (center_x2 - center_x1)**2 + (center_y2 - center_y1)**2
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_diag = (outer[:, 0] ** 2) + (outer[:, 1] ** 2)
    union = area1+area2-inter_area
    dious = inter_area / union - (inter_diag) / outer_diag
    dious = torch.clamp(dious,min=-1.0,max = 1.0)
    if exchange:
        dious = dious.T
    return dious

# 4.CIOU的提出

一个好的目标框回归损失应该考虑三个重要的几何因素：重叠面积、中心点距离、长宽比。
- GIoU：为了归一化坐标尺度，利用IoU，初步解决IoU为零的情况。
- DIoU：DIoU损失同时考虑了边界框的重叠面积和中心点距离。

anchor框和目标框之间的长宽比的一致性也是极其重要的

## 4.1.CIOU
$$
\begin{cases}
CIOU&=IOU-\frac{\rho(b,b^{gt})}{c^2}-\alpha \nu \\
\mathcal{L}_{CIOU}&=1-IOU+\frac{\rho(b,b^{gt})}{c^2} + \alpha \nu \\
\nu&=\frac{4}{\pi^2}(arctan\frac{\omega^{gt}}{h^{gt}}-arctan\frac{\omega}{h})^2 \\
\alpha&=\frac{\nu}{(1-IOU)+\nu}
\end{cases}
$$

CIOU比DIOU多出了$\alpha$和$\nu$这两个参数。其中$\alpha$是用于平衡比例的参数。$\nu$用来衡量anchor框和目标框之间的比例一致性。 从的定义式来看，损失函数会更加倾向于往重叠区域增多的方向优化，尤其是IOU为0的情况，这满足我们的要求。同时，在进行nms阶段，一般的评判标准是IOU，这个地方作者推荐替换为DIOU，这样考虑了中心点距离这一个信息，效果又有一定的提升。

In [ ]:
def bbox_overlaps_ciou(bboxes1, bboxes2):
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    cious = torch.zeros((rows, cols))
    if rows * cols == 0:
        return cious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        cious = torch.zeros((cols, rows))
        exchange = True

    w1 = bboxes1[:, 2] - bboxes1[:, 0]
    h1 = bboxes1[:, 3] - bboxes1[:, 1]
    w2 = bboxes2[:, 2] - bboxes2[:, 0]
    h2 = bboxes2[:, 3] - bboxes2[:, 1]

    area1 = w1 * h1
    area2 = w2 * h2

    center_x1 = (bboxes1[:, 2] + bboxes1[:, 0]) / 2
    center_y1 = (bboxes1[:, 3] + bboxes1[:, 1]) / 2
    center_x2 = (bboxes2[:, 2] + bboxes2[:, 0]) / 2
    center_y2 = (bboxes2[:, 3] + bboxes2[:, 1]) / 2

    inter_max_xy = torch.min(bboxes1[:, 2:],bboxes2[:, 2:])
    inter_min_xy = torch.max(bboxes1[:, :2],bboxes2[:, :2])
    out_max_xy = torch.max(bboxes1[:, 2:],bboxes2[:, 2:])
    out_min_xy = torch.min(bboxes1[:, :2],bboxes2[:, :2])

    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[:, 0] * inter[:, 1]
    inter_diag = (center_x2 - center_x1)**2 + (center_y2 - center_y1)**2
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_diag = (outer[:, 0] ** 2) + (outer[:, 1] ** 2)
    union = area1+area2-inter_area
    u = (inter_diag) / outer_diag
    iou = inter_area / union
    with torch.no_grad():
        arctan = torch.atan(w2 / h2) - torch.atan(w1 / h1)
        v = (4 / (math.pi ** 2)) * torch.pow((torch.atan(w2 / h2) - torch.atan(w1 / h1)), 2)
        S = 1 - iou
        alpha = v / (S + v)
        w_temp = 2 * w1
    ar = (8 / (math.pi ** 2)) * arctan * ((w1 - w_temp) * h1)
    cious = iou - (u + alpha * ar)
    cious = torch.clamp(cious,min=-1.0,max = 1.0)
    if exchange:
        cious = cious.T
    return cious